# Getting started with DeepMatcher

Note: you can run **[this notebook live in Google Colab](https://colab.research.google.com/github/anhaidgroup/deepmatcher/blob/master/examples/getting_started.ipynb)** and use free GPUs provided by Google.

This tutorial describes how to effortlessly perform entity matching using deep neural networks. Specifically, we will see how to match pairs of tuples (also called data records or table rows) to determine if they refer to the same real world entity. To do so, we will need labeled examples as input, i.e., tuple pairs which have been annotated as matches or non-matches. This will be used to train our neural network using supervised learning. At the end of this tutorial, you will have a trained neural network as output which you can easily apply to unlabeled tuple pairs to make predictions.

In [ ]:
'''from google.colab import drive
drive.mount('/content/drive')'''

"from google.colab import drive\ndrive.mount('/content/drive')"

As an overview, here are the 4 steps to use `deepmatcher` which we will go through in this tutorial:

<ol start="0">
  <li>Setup</li>
  <li>Process labeled data</li>
  <li>Define neural network model</li>
  <li>Train model</li>
  <li>Apply model to new data</li>
</ol>

Let's begin!

## Step 0. Setup

If you are running this notebook inside Colab, you will first need to install necessary packages by running the code below:

In [15]:
try:
    import deepmatcher
except:
    !pip install -qqq deepmatcher

In [16]:
# contorna erro legacy
!pip install torchtext==0.10.0

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


Now let's import `deepmatcher` which will do all the heavy lifting to build and train neural network models for entity matching. 

In [17]:
import pandas as pd
import deepmatcher as dm

We recommend having a GPU available for the training in Step 4. In case a GPU is not available, we will use all available CPU cores. You can run the following command to determine if a GPU is available and will be used for training:

In [18]:
import torch
torch.cuda.is_available()

True

### Download sample data for entity matching

Now let's get some sample data to play with in this tutorial. We will need three sets of labeled data and one set of unlabeled data:

1. **Training Data:** This is used for training our neural network model.
2. **Validation Data:** This is used for determining the configuration (i.e., hyperparameters) of our model in such a way that the model does not overfit to the training set.
3. **Test Data:** This is used to estimate the performance of our trained model on unlabeled data.
4. **Unlabeled Data:** The trained model is applied on this data to obtain predictions, which can then be used for downstream tasks in practical application scenarios.

We download these four data sets to the `sample_data` directory:

In [19]:
#!mkdir -p /content/drive/MyDrive/IC/unirTeste
#!wget -qnc -P /content/drive/MyDrive/IC/unirTeste/ http://pages.cs.wisc.edu/~anhai/data1/deepmatcher_data/Structured.zip

In [20]:
#!unzip -q /content/drive/MyDrive/IC/unirTeste/Structured.zip -d /content/drive/MyDrive/IC/unirTeste

In [21]:
#!ls /content/drive/MyDrive/IC/unirTeste/Structured/DBLP-GoogleScholar

In [22]:
'''
def merge_data(labeled, table_a, table_b, output):
  merged_csv = pd.read_csv(labeled).rename(columns={'ltable_id': 'left_id', 'rtable_id': 'right_id'})
  table_a_csv = pd.read_csv(table_a)
  table_a_csv = table_a_csv.rename(columns={col: 'left_' + col for col in table_a_csv.columns})
  table_b_csv = pd.read_csv(table_b)
  table_b_csv = table_b_csv.rename(columns={col: 'right_' + col for col in table_b_csv.columns})
  merged_csv = pd.merge(merged_csv, table_a_csv, on='left_id')
  merged_csv = pd.merge(merged_csv, table_b_csv, on='right_id')
  merged_csv['id'] = merged_csv[['left_id', 'right_id']].apply(lambda row: '_'.join([str(c) for c in row]), axis=1)
  del merged_csv['left_id']
  del merged_csv['right_id']
  merged_csv.to_csv(output, index=False)
'''

"\ndef merge_data(labeled, table_a, table_b, output):\n  merged_csv = pd.read_csv(labeled).rename(columns={'ltable_id': 'left_id', 'rtable_id': 'right_id'})\n  table_a_csv = pd.read_csv(table_a)\n  table_a_csv = table_a_csv.rename(columns={col: 'left_' + col for col in table_a_csv.columns})\n  table_b_csv = pd.read_csv(table_b)\n  table_b_csv = table_b_csv.rename(columns={col: 'right_' + col for col in table_b_csv.columns})\n  merged_csv = pd.merge(merged_csv, table_a_csv, on='left_id')\n  merged_csv = pd.merge(merged_csv, table_b_csv, on='right_id')\n  merged_csv['id'] = merged_csv[['left_id', 'right_id']].apply(lambda row: '_'.join([str(c) for c in row]), axis=1)\n  del merged_csv['left_id']\n  del merged_csv['right_id']\n  merged_csv.to_csv(output, index=False)\n"

In [23]:
'''
merge_data(
    '/content/drive/MyDrive/IC/unirTeste/Structured/DBLP-GoogleScholar/train.csv', 
    '/content/drive/MyDrive/IC/unirTeste/Structured/DBLP-GoogleScholar/tableA.csv', 
    '/content/drive/MyDrive/IC/unirTeste/Structured/DBLP-GoogleScholar/tableB.csv', 
    '/content/drive/MyDrive/IC/unirTeste/Structured/DBLP-GoogleScholar/joined_train.csv')
merge_data(
    '/content/drive/MyDrive/IC/unirTeste/Structured/DBLP-GoogleScholar/valid.csv', 
    '/content/drive/MyDrive/IC/unirTeste/Structured/DBLP-GoogleScholar/tableA.csv', 
    '/content/drive/MyDrive/IC/unirTeste/Structured/DBLP-GoogleScholar/tableB.csv', 
    '/content/drive/MyDrive/IC/unirTeste/Structured/DBLP-GoogleScholar/joined_valid.csv')
merge_data(
    '/content/drive/MyDrive/IC/unirTeste/Structured/DBLP-GoogleScholar/test.csv', 
    '/content/drive/MyDrive/IC/unirTeste/Structured/DBLP-GoogleScholar/tableA.csv', 
    '/content/drive/MyDrive/IC/unirTeste/Structured/DBLP-GoogleScholar/tableB.csv', 
    '/content/drive/MyDrive/IC/unirTeste/Structured/DBLP-GoogleScholar/joined_test.csv')
'''

"\nmerge_data(\n    '/content/drive/MyDrive/IC/unirTeste/Structured/DBLP-GoogleScholar/train.csv', \n    '/content/drive/MyDrive/IC/unirTeste/Structured/DBLP-GoogleScholar/tableA.csv', \n    '/content/drive/MyDrive/IC/unirTeste/Structured/DBLP-GoogleScholar/tableB.csv', \n    '/content/drive/MyDrive/IC/unirTeste/Structured/DBLP-GoogleScholar/joined_train.csv')\nmerge_data(\n    '/content/drive/MyDrive/IC/unirTeste/Structured/DBLP-GoogleScholar/valid.csv', \n    '/content/drive/MyDrive/IC/unirTeste/Structured/DBLP-GoogleScholar/tableA.csv', \n    '/content/drive/MyDrive/IC/unirTeste/Structured/DBLP-GoogleScholar/tableB.csv', \n    '/content/drive/MyDrive/IC/unirTeste/Structured/DBLP-GoogleScholar/joined_valid.csv')\nmerge_data(\n    '/content/drive/MyDrive/IC/unirTeste/Structured/DBLP-GoogleScholar/test.csv', \n    '/content/drive/MyDrive/IC/unirTeste/Structured/DBLP-GoogleScholar/tableA.csv', \n    '/content/drive/MyDrive/IC/unirTeste/Structured/DBLP-GoogleScholar/tableB.csv', \n    '/

To get an idea of how our data looks like, let's take a peek at the training dataset:

In [24]:
#pd.read_csv('/content/drive/MyDrive/IC/ICReteste/Deepmatcher/TesteErros/TesteErrosLabel/Erros6_10/Datasets/Structured/DBLP-GoogleScholar/joined_train.csv').head()

## Step 1. Process labeled data

Before we can use our data for training, `deepmatcher` needs to first load and process it in order to prepare it for neural network training. Currently `deepmatcher` only supports processing CSV files. Each CSV file is assumed to have the following kinds of columns:

* **"Left" attributes (required):** Our goal is to match tuple pairs. "Left" attributes are columns that correspond to the "left" tuple or the first tuple in the tuple pair. These column names are expected to be prefixed with "left_" by default.
* **"Right" attributes (required):** "Right" attributes are columns that correspond to the "right" tuple or the second tuple in the tuple pair. These column names are expected to be prefixed with "right_" by default.
* **Label column (required for train, validation, test):** Column containing the labels (match or non-match) for each tuple pair. Expected to be named "label" by default
* **ID column (required):** Column containing a unique ID for each tuple pair. This is for evaluation convenience.  Expected to be named "id" by default.

More details on what data processing involves and ways to customize it are described in **[this notebook](https://nbviewer.jupyter.org/github/anhaidgroup/deepmatcher/blob/master/examples/data_processing.ipynb)**. 

### Processing labeled data
In order to process our train, validation and test CSV files we call `dm.data.process` in the following code snippet which will load and process the CSV files and return three processed `MatchingDataset` objects respectively. These dataset objects will later be used for training and evaluation. The basic parameters to `dm.data.process` are as follows:

* **path (required): ** The path where all data is stored. This includes train, validation and test. `deepmatcher` may create new files in this directory to store information about these data sets. This allows subsequent `dm.data.process` calls to be much faster.
* **train (required): ** File name of training data in `path` directory.
* **validation (required): ** File name of validation data in `path` directory.
* **test (optional): ** File name of test data in `path` directory.
* **ignore_columns (optional): ** Any columns in the CSV files that you may want to ignore for the purposes of training. These should be included here. 

Note that the train, validation and test CSVs must all share the same schema, i.e., they should have the same columns. Processing data involves several steps and can take several minutes to complete, especially if this is the first time you are running the `deepmatcher` package.

NOTE: If you are running this in Colab, you may get a message saying 'Memory usage is close to the limit.' You can safely ignore it for now. We are working on reducing the memory footprint.

In [ ]:
!wget https://dl.fbaipublicfiles.com/fasttext/vectors-wiki/wiki.en.zip --directory-prefix=/root/.vector_cache
!unzip /root/.vector_cache/wiki.en.zip -d /root/.vector_cache/
!rm /root/.vector_cache/wiki.en.vec

--2022-08-09 20:51:22--  https://dl.fbaipublicfiles.com/fasttext/vectors-wiki/wiki.en.zip
Resolving dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)... 104.22.75.142, 104.22.74.142, 172.67.9.4, ...
Connecting to dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)|104.22.75.142|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10356881291 (9.6G) [application/zip]
Saving to: ‘/root/.vector_cache/wiki.en.zip’

wiki.en.zip         100%[===================>]   9.65G  17.6MB/s    in 9m 7s   

2022-08-09 21:00:30 (18.1 MB/s) - ‘/root/.vector_cache/wiki.en.zip’ saved [10356881291/10356881291]

Archive:  /root/.vector_cache/wiki.en.zip
  inflating: /root/.vector_cache/wiki.en.vec  
  inflating: /root/.vector_cache/wiki.en.bin  


In [ ]:
'''train, validation, test = dm.data.process(
    path='/content/drive/MyDrive/IC/ICReteste/Deepmatcher/TesteErros/TesteErrosLabel/Erros6_10/Datasets/Dirty/DBLP-GoogleScholar/',
    train='joined_train.csv',
    validation='joined_valid.csv',
    test='joined_test_error.csv')'''

"train, validation, test = dm.data.process(\n    path='/content/drive/MyDrive/IC/ICReteste/Deepmatcher/TesteErros/TesteErrosLabel/Erros6_10/Datasets/Dirty/DBLP-GoogleScholar/',\n    train='joined_train.csv',\n    validation='joined_valid.csv',\n    test='joined_test_error.csv')"

#### Peeking at processed data
Let's take a look at how the processed data looks like. To do this, we get the raw `pandas` table corresponding to the processed training dataset object. 

In [ ]:
'''train_table = train.get_raw_table()
train_table.head()'''

'train_table = train.get_raw_table()\ntrain_table.head()'

The processed attribute values have been tokenized and lowercased so they may not look exactly the same as the input training data. These modifications help the neural network generalize better, i.e., perform better on data not trained on. 

## Step 2. Define neural network model

In this step you tell `deepmatcher` what kind of neural network you would like to use for entity matching. The easiest way to do this is to use one of the several kinds of neural network models that comes built-in with `deepmatcher`. To use a built-in network, construct a `dm.MatchingModel` as follows:

`model = dm.MatchingModel(attr_summarizer='<TYPE>')`

where `<TYPE>` is one of `sif`, `rnn`, `attention` or `hybrid`. If you are not familiar with what these mean, we strongly recommend taking a look at either **[slides from our talk on deepmatcher](http://bit.do/deepmatcher-talk)** for a high level overview, or **[our paper](http://pages.cs.wisc.edu/~anhai/papers1/deepmatcher-sigmod18.pdf)** for a more detailed explanation. Here we give briefly describe the intuition behind these four model types:
* **sif:** This model considers the **words** present in each attribute value pair to determine a match or non-match. It does not take word order into account.
* **rnn:** This model considers the **sequences of words** present in each attribute value pair to determine a match or non-match.
* **attention:** This model considers the **alignment of words** present in each attribute value pair to determine a match or non-match. It does not take word order into account.
* **hybrid:** This model considers the **alignment of sequences of words** present in each attribute value pair to determine a match or non-match. This is the default.

`deepmatcher` is highly customizable and allows you to tune almost every aspect of the neural network model for your application scenario. **[This tutorial](https://nbviewer.jupyter.org/github/anhaidgroup/deepmatcher/blob/master/examples/matching_models.ipynb)** discusses the structure of `MatchingModel`s and how they can be customized.

For this tutorial, let's create a `hybrid` model for entity matching:

In [ ]:
'''model = dm.MatchingModel(attr_summarizer='attention')'''

"model = dm.MatchingModel(attr_summarizer='attention')"

## Step 3. Train model

Next, we train the defined neural network model using the processed training and validation data. To do so, we call the `run_train` method which takes the following basic parameters:

* **train:** The processed training dataset object (of type `MatchingDataset`).
* **validation:** The processed validation dataset object (of type `MatchingDataset`).
* **epochs:** Number of times to go over the entire `train` data for training the model.
* **batch_size:** Number of labeled examples (tuple pairs) to use for each training step. This value may be increased if you have a lot of training data and would like to speed up training. The optimal value is dataset dependent.
* **best_save_path:** Path to save the best model.
* **pos_neg_ratio**: The ratio of the weight of positive examples (matches) to weight of negative examples (non-matches). This value should be increased if you have fewer matches than non-matches in your data. The optimal value is dataset dependent.

Many other aspects of the training algorithm can be customized. For details on this, please refer the API documentation for **[run_train]()**

In [ ]:
'''model.run_train(
    train,
    validation,
    epochs=15,
    batch_size=32,
    best_save_path='attention_model.pth',
    pos_neg_ratio=3)'''

"model.run_train(\n    train,\n    validation,\n    epochs=15,\n    batch_size=32,\n    best_save_path='attention_model.pth',\n    pos_neg_ratio=3)"

## Novos Testes

In [ ]:
!git clone https://github.com/pauloh48/IC.git

Cloning into 'IC'...
remote: Enumerating objects: 985, done.
remote: Counting objects: 100% (55/55), done.
remote: Compressing objects: 100% (51/51), done.
remote: Total 985 (delta 16), reused 7 (delta 0), pack-reused 930
Receiving objects: 100% (985/985), 252.10 MiB | 17.86 MiB/s, done.
Resolving deltas: 100% (600/600), done.
Checking out files: 100% (1274/1274), done.


### RNN

#### STRUCTURED

##### Amazon-Google

In [ ]:
train, validation, test = dm.data.process(
    path='/content/IC/datasesErros/CMedium_30_3_5/Structured/Amazon-Google/',
    train='joined_train.csv',
    validation='joined_valid.csv',
    test='joined_test.csv')


Reading and processing data from "/content/IC/datasesErros/CMedium_30_3_5/Structured/Amazon-Google/joined_train.csv"
0% [############################# ] 100% | ETA: 00:00:00
Reading and processing data from "/content/IC/datasesErros/CMedium_30_3_5/Structured/Amazon-Google/joined_valid.csv"
0% [############################# ] 100% | ETA: 00:00:00
Reading and processing data from "/content/IC/datasesErros/CMedium_30_3_5/Structured/Amazon-Google/joined_test.csv"
0% [############################# ] 100% | ETA: 00:00:00Warning : `load_model` does not return WordVectorModel or SupervisedModel any more, but a `FastText` object which is very similar.

Building vocabulary
0% [#######] 100% | ETA: 00:00:00
Total time elapsed: 00:00:00

Computing principal components
0% [#######] 100% | ETA: 00:00:00
Total time elapsed: 00:00:01


In [ ]:
model = dm.MatchingModel(attr_summarizer='rnn')

In [ ]:
model.run_train(
    train,
    validation,
    epochs=15,
    batch_size=32,
    best_save_path='rnn_model.pth',
    pos_neg_ratio=3)

/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:949: UserWarning: Using non-full backward hooks on a Module that does not take as input a single Tensor or a tuple of Tensors is deprecated and will be removed in future versions. This hook will be missing some of the grad_input. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using non-full backward hooks on a Module that does not take as input a "
/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:974: UserWarning: Using a non-full backward hook when the forward contains multiple autograd Nodes is deprecated and will be removed in future versions. This hook will be missing some grad_input. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using a non-full backward hook when the forward contains multiple autograd Nodes "


* Number of trainable parameters: 1762802
===>  TRAIN Epoch 1


/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:939: UserWarning: Using non-full backward hooks on a Module that does not return a single Tensor or a tuple of Tensors is deprecated and will be removed in future versions. This hook will be missing some of the grad_output. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using non-full backward hooks on a Module that does not return a "
/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:2742: UserWarning: reduction: 'mean' divides the total loss by both the batch size and the support size.'batchmean' divides only by the batch size, and aligns with the KL div math definition.'mean' will be changed to behave the same as 'batchmean' in the next major release.
  "reduction: 'mean' divides the total loss by both the batch size and the support size."
0% [██████████████████████████████] 100% | ETA: 00:00:00
Total time elapsed: 00:00:10


Finished Epoch 1 || Run Time:    6.1 | Load Time:    4.7 || F1:  22.83 | Prec:  35.89 | Rec:  16.74 || Ex/s: 637.13

===>  EVAL Epoch 1


0% [██████████████] 100% | ETA: 00:00:00
Total time elapsed: 00:00:02


...

0% [██████████████████████████████] 100% | ETA: 00:00:00
Total time elapsed: 00:00:10


Finished Epoch 15 || Run Time:    5.9 | Load Time:    4.8 || F1:  99.07 | Prec:  98.58 | Rec:  99.57 || Ex/s: 640.38

===>  EVAL Epoch 15


0% [██████████████] 100% | ETA: 00:00:00
Total time elapsed: 00:00:02


Finished Epoch 15 || Run Time:    0.9 | Load Time:    1.5 || F1:  42.64 | Prec:  52.50 | Rec:  35.90 || Ex/s: 980.62

---------------------

Loading best model...
Training done.


tensor(47.3988, device='cuda:0')

In [ ]:
# Compute F1 on test set
model.run_eval(test)

===>  EVAL Epoch 2


/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:949: UserWarning: Using non-full backward hooks on a Module that does not take as input a single Tensor or a tuple of Tensors is deprecated and will be removed in future versions. This hook will be missing some of the grad_input. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using non-full backward hooks on a Module that does not take as input a "
/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:939: UserWarning: Using non-full backward hooks on a Module that does not return a single Tensor or a tuple of Tensors is deprecated and will be removed in future versions. This hook will be missing some of the grad_output. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using non-full backward hooks on a Module that does not return a "
/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:974: UserWarning: Using a non-f

Finished Epoch 2 || Run Time:    0.9 | Load Time:    1.5 || F1:  49.03 | Prec:  44.72 | Rec:  54.27 || Ex/s: 963.79



tensor(49.0347, device='cuda:0')

##### Beer

In [ ]:
train, validation, test = dm.data.process(
    path='/content/IC/datasesErros/CMedium_30_3_5/Structured/Beer/',
    train='joined_train.csv',
    validation='joined_valid.csv',
    test='joined_test.csv')


Reading and processing data from "/content/IC/datasesErros/CMedium_30_3_5/Structured/Beer/joined_train.csv"
0% [############################# ] 100% | ETA: 00:00:00
Reading and processing data from "/content/IC/datasesErros/CMedium_30_3_5/Structured/Beer/joined_valid.csv"
0% [############################# ] 100% | ETA: 00:00:00
Reading and processing data from "/content/IC/datasesErros/CMedium_30_3_5/Structured/Beer/joined_test.csv"
0% [############################# ] 100% | ETA: 00:00:00
Building vocabulary
0% [#] 100% | ETA: 00:00:00
Total time elapsed: 00:00:00

Computing principal components
0% [#] 100% | ETA: 00:00:00
Total time elapsed: 00:00:00


In [ ]:
model = dm.MatchingModel(attr_summarizer='rnn')

In [ ]:
model.run_train(
    train,
    validation,
    epochs=15,
    batch_size=32,
    best_save_path='rnn_model.pth',
    pos_neg_ratio=3)

* Number of trainable parameters: 2169602
===>  TRAIN Epoch 1


/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:949: UserWarning: Using non-full backward hooks on a Module that does not take as input a single Tensor or a tuple of Tensors is deprecated and will be removed in future versions. This hook will be missing some of the grad_input. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using non-full backward hooks on a Module that does not take as input a "
/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:974: UserWarning: Using a non-full backward hook when the forward contains multiple autograd Nodes is deprecated and will be removed in future versions. This hook will be missing some grad_input. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using a non-full backward hook when the forward contains multiple autograd Nodes "
/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:939: UserWarning: Using non-full backward h

Finished Epoch 1 || Run Time:    0.3 | Load Time:    0.3 || F1:   0.00 | Prec:   0.00 | Rec:   0.00 || Ex/s: 416.87

===>  EVAL Epoch 1
Finished Epoch 1 || Run Time:    0.1 | Load Time:    0.1 || F1:   0.00 | Prec:   0.00 | Rec:   0.00 || Ex/s: 545.90

* Best F1: tensor(0., device='cuda:0')
Saving best model...
Done.
---------------------

===>  TRAIN Epoch 2


0% [█] 100% | ETA: 00:00:00
Total time elapsed: 00:00:00


Finished Epoch 2 || Run Time:    0.3 | Load Time:    0.3 || F1:  56.67 | Prec:  85.00 | Rec:  42.50 || Ex/s: 432.68

===>  EVAL Epoch 2
Finished Epoch 2 || Run Time:    0.0 | Load Time:    0.1 || F1:  60.61 | Prec:  52.63 | Rec:  71.43 || Ex/s: 616.88

* Best F1: tensor(60.6061, device='cuda:0')
Saving best model...
Done.
---------------------

===>  TRAIN Epoch 3


...

0% [█] 100% | ETA: 00:00:00
Total time elapsed: 00:00:00


Finished Epoch 14 || Run Time:    0.3 | Load Time:    0.3 || F1: 100.00 | Prec: 100.00 | Rec: 100.00 || Ex/s: 440.30

===>  EVAL Epoch 14
Finished Epoch 14 || Run Time:    0.0 | Load Time:    0.1 || F1:  45.45 | Prec:  33.33 | Rec:  71.43 || Ex/s: 622.56

---------------------

===>  TRAIN Epoch 15


0% [█] 100% | ETA: 00:00:00
Total time elapsed: 00:00:00


Finished Epoch 15 || Run Time:    0.3 | Load Time:    0.3 || F1: 100.00 | Prec: 100.00 | Rec: 100.00 || Ex/s: 431.10

===>  EVAL Epoch 15
Finished Epoch 15 || Run Time:    0.0 | Load Time:    0.1 || F1:  45.45 | Prec:  33.33 | Rec:  71.43 || Ex/s: 621.71

---------------------

Loading best model...
Training done.


tensor(60.6061, device='cuda:0')

In [ ]:
# Compute F1 on test set
model.run_eval(test)

===>  EVAL Epoch 2


/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:949: UserWarning: Using non-full backward hooks on a Module that does not take as input a single Tensor or a tuple of Tensors is deprecated and will be removed in future versions. This hook will be missing some of the grad_input. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using non-full backward hooks on a Module that does not take as input a "
/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:939: UserWarning: Using non-full backward hooks on a Module that does not return a single Tensor or a tuple of Tensors is deprecated and will be removed in future versions. This hook will be missing some of the grad_output. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using non-full backward hooks on a Module that does not return a "
/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:974: UserWarning: Using a non-f

Finished Epoch 2 || Run Time:    0.0 | Load Time:    0.1 || F1:  52.63 | Prec:  41.67 | Rec:  71.43 || Ex/s: 606.83



tensor(52.6316, device='cuda:0')

##### DBLP-ACM

In [ ]:
train, validation, test = dm.data.process(
    path='/content/IC/datasesErros/CMedium_30_3_5/Structured/DBLP-ACM/',
    train='joined_train.csv',
    validation='joined_valid.csv',
    test='joined_test.csv')

In [ ]:
model = dm.MatchingModel(attr_summarizer='rnn')

In [ ]:
model.run_train(
    train,
    validation,
    epochs=15,
    batch_size=32,
    best_save_path='rnn_model.pth',
    pos_neg_ratio=3)

/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:949: UserWarning: Using non-full backward hooks on a Module that does not take as input a single Tensor or a tuple of Tensors is deprecated and will be removed in future versions. This hook will be missing some of the grad_input. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using non-full backward hooks on a Module that does not take as input a "
/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:974: UserWarning: Using a non-full backward hook when the forward contains multiple autograd Nodes is deprecated and will be removed in future versions. This hook will be missing some grad_input. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using a non-full backward hook when the forward contains multiple autograd Nodes "


* Number of trainable parameters: 2169602
===>  TRAIN Epoch 1


/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:939: UserWarning: Using non-full backward hooks on a Module that does not return a single Tensor or a tuple of Tensors is deprecated and will be removed in future versions. This hook will be missing some of the grad_output. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using non-full backward hooks on a Module that does not return a "
/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:2742: UserWarning: reduction: 'mean' divides the total loss by both the batch size and the support size.'batchmean' divides only by the batch size, and aligns with the KL div math definition.'mean' will be changed to behave the same as 'batchmean' in the next major release.
  "reduction: 'mean' divides the total loss by both the batch size and the support size."
0% [██████████████████████████████] 100% | ETA: 00:00:00
Total time elapsed: 00:00:24


Finished Epoch 1 || Run Time:   10.7 | Load Time:   14.2 || F1:  89.38 | Prec:  85.83 | Rec:  93.24 || Ex/s: 297.84

===>  EVAL Epoch 1


0% [███████████████] 100% | ETA: 00:00:00
Total time elapsed: 00:00:04


...

0% [██████████████████████████████] 100% | ETA: 00:00:00
Total time elapsed: 00:00:20


Finished Epoch 15 || Run Time:    9.0 | Load Time:   11.8 || F1:  99.74 | Prec:  99.48 | Rec: 100.00 || Ex/s: 357.00

===>  EVAL Epoch 15


0% [███████████████] 100% | ETA: 00:00:00
Total time elapsed: 00:00:04


Finished Epoch 15 || Run Time:    1.3 | Load Time:    3.7 || F1:  97.03 | Prec:  94.84 | Rec:  99.32 || Ex/s: 487.95

---------------------

Loading best model...
Training done.


tensor(98.3165, device='cuda:0')

In [ ]:
# Compute F1 on test set
model.run_eval(test)

===>  EVAL Epoch 2


/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:949: UserWarning: Using non-full backward hooks on a Module that does not take as input a single Tensor or a tuple of Tensors is deprecated and will be removed in future versions. This hook will be missing some of the grad_input. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using non-full backward hooks on a Module that does not take as input a "
/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:939: UserWarning: Using non-full backward hooks on a Module that does not return a single Tensor or a tuple of Tensors is deprecated and will be removed in future versions. This hook will be missing some of the grad_output. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using non-full backward hooks on a Module that does not return a "
/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:974: UserWarning: Using a non-f

Finished Epoch 2 || Run Time:    1.4 | Load Time:    3.7 || F1:  96.77 | Prec:  95.81 | Rec:  97.75 || Ex/s: 492.84



tensor(96.7670, device='cuda:0')

##### DBLP-GoogleScholar

In [ ]:
train, validation, test = dm.data.process(
    path='/content/IC/datasesErros/CMedium_30_3_5/Structured/DBLP-GoogleScholar/',
    train='joined_train.csv',
    validation='joined_valid.csv',
    test='joined_test.csv')

In [ ]:
model = dm.MatchingModel(attr_summarizer='rnn')

In [ ]:
model.run_train(
    train,
    validation,
    epochs=15,
    batch_size=32,
    best_save_path='rnn_model.pth',
    pos_neg_ratio=3)

/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:949: UserWarning: Using non-full backward hooks on a Module that does not take as input a single Tensor or a tuple of Tensors is deprecated and will be removed in future versions. This hook will be missing some of the grad_input. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using non-full backward hooks on a Module that does not take as input a "
/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:974: UserWarning: Using a non-full backward hook when the forward contains multiple autograd Nodes is deprecated and will be removed in future versions. This hook will be missing some grad_input. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using a non-full backward hook when the forward contains multiple autograd Nodes "


* Number of trainable parameters: 2169602
===>  TRAIN Epoch 1


/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:939: UserWarning: Using non-full backward hooks on a Module that does not return a single Tensor or a tuple of Tensors is deprecated and will be removed in future versions. This hook will be missing some of the grad_output. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using non-full backward hooks on a Module that does not return a "
/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:2742: UserWarning: reduction: 'mean' divides the total loss by both the batch size and the support size.'batchmean' divides only by the batch size, and aligns with the KL div math definition.'mean' will be changed to behave the same as 'batchmean' in the next major release.
  "reduction: 'mean' divides the total loss by both the batch size and the support size."
0% [██████████████████████████████] 100% | ETA: 00:00:00
Total time elapsed: 00:00:47


Finished Epoch 1 || Run Time:   21.6 | Load Time:   26.3 || F1:  78.59 | Prec:  72.50 | Rec:  85.81 || Ex/s: 359.65

===>  EVAL Epoch 1


0% [██████████████████████████████] 100% | ETA: 00:00:00
Total time elapsed: 00:00:10


...

0% [██████████████████████████████] 100% | ETA: 00:00:00
Total time elapsed: 00:00:44


Finished Epoch 15 || Run Time:   19.8 | Load Time:   24.6 || F1:  99.77 | Prec:  99.84 | Rec:  99.69 || Ex/s: 388.01

===>  EVAL Epoch 15


0% [██████████████████████████████] 100% | ETA: 00:00:00
Total time elapsed: 00:00:10


Finished Epoch 15 || Run Time:    3.0 | Load Time:    7.5 || F1:  91.02 | Prec:  91.58 | Rec:  90.47 || Ex/s: 546.07

---------------------

Loading best model...
Training done.


tensor(91.2821, device='cuda:0')

In [ ]:
# Compute F1 on test set
model.run_eval(test)

===>  EVAL Epoch 6


/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:949: UserWarning: Using non-full backward hooks on a Module that does not take as input a single Tensor or a tuple of Tensors is deprecated and will be removed in future versions. This hook will be missing some of the grad_input. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using non-full backward hooks on a Module that does not take as input a "
/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:939: UserWarning: Using non-full backward hooks on a Module that does not return a single Tensor or a tuple of Tensors is deprecated and will be removed in future versions. This hook will be missing some of the grad_output. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using non-full backward hooks on a Module that does not return a "
/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:974: UserWarning: Using a non-f

Finished Epoch 6 || Run Time:    3.0 | Load Time:    7.6 || F1:  91.10 | Prec:  91.35 | Rec:  90.84 || Ex/s: 543.03



tensor(91.0965, device='cuda:0')

##### Walmart-Amazon

In [ ]:
train, validation, test = dm.data.process(
    path='/content/IC/datasesErros/CMedium_30_3_5/Structured/Walmart-Amazon/',
    train='joined_train.csv',
    validation='joined_valid.csv',
    test='joined_test.csv')

In [ ]:
model = dm.MatchingModel(attr_summarizer='rnn')

In [ ]:
model.run_train(
    train,
    validation,
    epochs=15,
    batch_size=32,
    best_save_path='rnn_model.pth',
    pos_neg_ratio=3)

* Number of trainable parameters: 2576402
===>  TRAIN Epoch 1


/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:949: UserWarning: Using non-full backward hooks on a Module that does not take as input a single Tensor or a tuple of Tensors is deprecated and will be removed in future versions. This hook will be missing some of the grad_input. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using non-full backward hooks on a Module that does not take as input a "
/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:974: UserWarning: Using a non-full backward hook when the forward contains multiple autograd Nodes is deprecated and will be removed in future versions. This hook will be missing some grad_input. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using a non-full backward hook when the forward contains multiple autograd Nodes "
/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:939: UserWarning: Using non-full backward h

Finished Epoch 1 || Run Time:    7.7 | Load Time:    6.6 || F1:  28.43 | Prec:  52.83 | Rec:  19.44 || Ex/s: 430.81

===>  EVAL Epoch 1


0% [█████████████] 100% | ETA: 00:00:00
Total time elapsed: 00:00:03


Finished Epoch 1 || Run Time:    1.2 | Load Time:    2.1 || F1:  57.22 | Prec:  60.34 | Rec:  54.40 || Ex/s: 627.48

* Best F1: tensor(57.2207, device='cuda:0')
Saving best model...
Done.
---------------------

===>  TRAIN Epoch 2


...

0% [██████████████████████████████] 100% | ETA: 00:00:00
Total time elapsed: 00:00:14


Finished Epoch 15 || Run Time:    7.8 | Load Time:    6.6 || F1: 100.00 | Prec: 100.00 | Rec: 100.00 || Ex/s: 426.68

===>  EVAL Epoch 15


0% [█████████████] 100% | ETA: 00:00:00
Total time elapsed: 00:00:03


Finished Epoch 15 || Run Time:    1.2 | Load Time:    2.1 || F1:  65.29 | Prec:  75.51 | Rec:  57.51 || Ex/s: 620.51

---------------------

Loading best model...
Training done.


tensor(67.5159, device='cuda:0')

In [ ]:
# Compute F1 on test set
model.run_eval(test)

===>  EVAL Epoch 6


/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:949: UserWarning: Using non-full backward hooks on a Module that does not take as input a single Tensor or a tuple of Tensors is deprecated and will be removed in future versions. This hook will be missing some of the grad_input. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using non-full backward hooks on a Module that does not take as input a "
/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:939: UserWarning: Using non-full backward hooks on a Module that does not return a single Tensor or a tuple of Tensors is deprecated and will be removed in future versions. This hook will be missing some of the grad_output. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using non-full backward hooks on a Module that does not return a "
/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:974: UserWarning: Using a non-f

Finished Epoch 6 || Run Time:    1.2 | Load Time:    2.1 || F1:  66.24 | Prec:  87.29 | Rec:  53.37 || Ex/s: 612.87



tensor(66.2379, device='cuda:0')

#### TEXTUAL

##### Abt-Buy

In [ ]:
train, validation, test = dm.data.process(
    path='/content/IC/datasesErros/CMedium_30_3_5/Textual/Abt-Buy/',
    train='joined_train.csv',
    validation='joined_valid.csv',
    test='joined_test.csv')


Reading and processing data from "/content/IC/datasesErros/CMedium_30_3_5/Textual/Abt-Buy/joined_train.csv"
0% [############################# ] 100% | ETA: 00:00:00
Reading and processing data from "/content/IC/datasesErros/CMedium_30_3_5/Textual/Abt-Buy/joined_valid.csv"
0% [############################# ] 100% | ETA: 00:00:00
Reading and processing data from "/content/IC/datasesErros/CMedium_30_3_5/Textual/Abt-Buy/joined_test.csv"
0% [############################# ] 100% | ETA: 00:00:00Warning : `load_model` does not return WordVectorModel or SupervisedModel any more, but a `FastText` object which is very similar.

Building vocabulary
0% [######] 100% | ETA: 00:00:00
Total time elapsed: 00:00:00

Computing principal components
0% [######] 100% | ETA: 00:00:00
Total time elapsed: 00:00:03


In [ ]:
model = dm.MatchingModel(attr_summarizer='rnn')

In [ ]:
model.run_train(
    train,
    validation,
    epochs=15,
    batch_size=32,
    best_save_path='rnn_model.pth',
    pos_neg_ratio=3)

/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:949: UserWarning: Using non-full backward hooks on a Module that does not take as input a single Tensor or a tuple of Tensors is deprecated and will be removed in future versions. This hook will be missing some of the grad_input. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using non-full backward hooks on a Module that does not take as input a "
/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:974: UserWarning: Using a non-full backward hook when the forward contains multiple autograd Nodes is deprecated and will be removed in future versions. This hook will be missing some grad_input. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using a non-full backward hook when the forward contains multiple autograd Nodes "


* Number of trainable parameters: 1762802
===>  TRAIN Epoch 1


/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:939: UserWarning: Using non-full backward hooks on a Module that does not return a single Tensor or a tuple of Tensors is deprecated and will be removed in future versions. This hook will be missing some of the grad_output. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using non-full backward hooks on a Module that does not return a "
/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:2742: UserWarning: reduction: 'mean' divides the total loss by both the batch size and the support size.'batchmean' divides only by the batch size, and aligns with the KL div math definition.'mean' will be changed to behave the same as 'batchmean' in the next major release.
  "reduction: 'mean' divides the total loss by both the batch size and the support size."
0% [██████████████████████████████] 100% | ETA: 00:00:00
Total time elapsed: 00:00:20


Finished Epoch 1 || Run Time:    8.0 | Load Time:   12.2 || F1:  16.63 | Prec:  29.83 | Rec:  11.53 || Ex/s: 284.03

===>  EVAL Epoch 1


0% [████████████] 100% | ETA: 00:00:00
Total time elapsed: 00:00:03


...

0% [██████████████████████████████] 100% | ETA: 00:00:00
Total time elapsed: 00:00:17


Finished Epoch 15 || Run Time:    6.5 | Load Time:   10.5 || F1:  99.44 | Prec:  98.88 | Rec: 100.00 || Ex/s: 336.94

===>  EVAL Epoch 15


0% [████████████] 100% | ETA: 00:00:00
Total time elapsed: 00:00:04


Finished Epoch 15 || Run Time:    1.0 | Load Time:    3.1 || F1:  31.98 | Prec:  39.86 | Rec:  26.70 || Ex/s: 468.26

---------------------

Loading best model...
Training done.


tensor(37.5510, device='cuda:0')

In [ ]:
# Compute F1 on test set
model.run_eval(test)

===>  EVAL Epoch 5


/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:949: UserWarning: Using non-full backward hooks on a Module that does not take as input a single Tensor or a tuple of Tensors is deprecated and will be removed in future versions. This hook will be missing some of the grad_input. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using non-full backward hooks on a Module that does not take as input a "
/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:939: UserWarning: Using non-full backward hooks on a Module that does not return a single Tensor or a tuple of Tensors is deprecated and will be removed in future versions. This hook will be missing some of the grad_output. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using non-full backward hooks on a Module that does not return a "
/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:974: UserWarning: Using a non-f

Finished Epoch 5 || Run Time:    0.9 | Load Time:    3.1 || F1:  40.00 | Prec:  35.91 | Rec:  45.15 || Ex/s: 478.64



tensor(40., device='cuda:0')

#### DIRTY

##### DBLP-ACM

In [11]:
train, validation, test = dm.data.process(
    path='/content/IC/datasesErros/CMedium_30_3_5/Dirty/DBLP-ACM/',
    train='joined_train.csv',
    validation='joined_valid.csv',
    test='joined_test.csv')


Reading and processing data from "/content/IC/datasesErros/CMedium_30_3_5/Dirty/DBLP-ACM/joined_train.csv"
0% [############################# ] 100% | ETA: 00:00:00
Reading and processing data from "/content/IC/datasesErros/CMedium_30_3_5/Dirty/DBLP-ACM/joined_valid.csv"
0% [############################# ] 100% | ETA: 00:00:00
Reading and processing data from "/content/IC/datasesErros/CMedium_30_3_5/Dirty/DBLP-ACM/joined_test.csv"
0% [############################# ] 100% | ETA: 00:00:00Warning : `load_model` does not return WordVectorModel or SupervisedModel any more, but a `FastText` object which is very similar.

Building vocabulary
0% [########] 100% | ETA: 00:00:00
Total time elapsed: 00:00:00

Computing principal components
0% [########] 100% | ETA: 00:00:00
Total time elapsed: 00:00:04


In [12]:
model = dm.MatchingModel(attr_summarizer='rnn')

In [13]:
model.run_train(
    train,
    validation,
    epochs=15,
    batch_size=32,
    best_save_path='rnn_model.pth',
    pos_neg_ratio=3)

/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:949: UserWarning: Using non-full backward hooks on a Module that does not take as input a single Tensor or a tuple of Tensors is deprecated and will be removed in future versions. This hook will be missing some of the grad_input. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using non-full backward hooks on a Module that does not take as input a "
/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:974: UserWarning: Using a non-full backward hook when the forward contains multiple autograd Nodes is deprecated and will be removed in future versions. This hook will be missing some grad_input. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using a non-full backward hook when the forward contains multiple autograd Nodes "


* Number of trainable parameters: 2169602
===>  TRAIN Epoch 1


/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:939: UserWarning: Using non-full backward hooks on a Module that does not return a single Tensor or a tuple of Tensors is deprecated and will be removed in future versions. This hook will be missing some of the grad_output. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using non-full backward hooks on a Module that does not return a "
/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:2742: UserWarning: reduction: 'mean' divides the total loss by both the batch size and the support size.'batchmean' divides only by the batch size, and aligns with the KL div math definition.'mean' will be changed to behave the same as 'batchmean' in the next major release.
  "reduction: 'mean' divides the total loss by both the batch size and the support size."
0% [██████████████████████████████] 100% | ETA: 00:00:00
Total time elapsed: 00:00:26


Finished Epoch 1 || Run Time:   10.7 | Load Time:   15.5 || F1:  68.84 | Prec:  60.83 | Rec:  79.28 || Ex/s: 282.56

===>  EVAL Epoch 1


0% [███████████████] 100% | ETA: 00:00:00
Total time elapsed: 00:00:05


...

0% [██████████████████████████████] 100% | ETA: 00:00:00
Total time elapsed: 00:00:23


Finished Epoch 15 || Run Time:    9.5 | Load Time:   14.2 || F1:  99.85 | Prec:  99.92 | Rec:  99.77 || Ex/s: 312.77

===>  EVAL Epoch 15


0% [███████████████] 100% | ETA: 00:00:00
Total time elapsed: 00:00:05


Finished Epoch 15 || Run Time:    1.4 | Load Time:    4.5 || F1:  91.56 | Prec:  90.35 | Rec:  92.79 || Ex/s: 419.53

---------------------

Loading best model...
Training done.


tensor(91.7595, device='cuda:0')

In [14]:
# Compute F1 on test set
model.run_eval(test)

===>  EVAL Epoch 13


/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:949: UserWarning: Using non-full backward hooks on a Module that does not take as input a single Tensor or a tuple of Tensors is deprecated and will be removed in future versions. This hook will be missing some of the grad_input. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using non-full backward hooks on a Module that does not take as input a "
/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:939: UserWarning: Using non-full backward hooks on a Module that does not return a single Tensor or a tuple of Tensors is deprecated and will be removed in future versions. This hook will be missing some of the grad_output. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using non-full backward hooks on a Module that does not return a "
/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:974: UserWarning: Using a non-f

Finished Epoch 13 || Run Time:    1.4 | Load Time:    4.3 || F1:  90.99 | Prec:  88.84 | Rec:  93.24 || Ex/s: 429.77



tensor(90.9890, device='cuda:0')

##### DBLP-GoogleScholar

In [11]:
train, validation, test = dm.data.process(
    path='/content/IC/datasesErros/CMedium_30_3_5/Dirty/DBLP-GoogleScholar//',
    train='joined_train.csv',
    validation='joined_valid.csv',
    test='joined_test.csv')

In [12]:
model = dm.MatchingModel(attr_summarizer='rnn')

In [13]:
model.run_train(
    train,
    validation,
    epochs=15,
    batch_size=32,
    best_save_path='rnn_model.pth',
    pos_neg_ratio=3)

/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:949: UserWarning: Using non-full backward hooks on a Module that does not take as input a single Tensor or a tuple of Tensors is deprecated and will be removed in future versions. This hook will be missing some of the grad_input. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using non-full backward hooks on a Module that does not take as input a "
/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:974: UserWarning: Using a non-full backward hook when the forward contains multiple autograd Nodes is deprecated and will be removed in future versions. This hook will be missing some grad_input. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using a non-full backward hook when the forward contains multiple autograd Nodes "


* Number of trainable parameters: 2169602
===>  TRAIN Epoch 1


/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:939: UserWarning: Using non-full backward hooks on a Module that does not return a single Tensor or a tuple of Tensors is deprecated and will be removed in future versions. This hook will be missing some of the grad_output. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using non-full backward hooks on a Module that does not return a "
/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:2742: UserWarning: reduction: 'mean' divides the total loss by both the batch size and the support size.'batchmean' divides only by the batch size, and aligns with the KL div math definition.'mean' will be changed to behave the same as 'batchmean' in the next major release.
  "reduction: 'mean' divides the total loss by both the batch size and the support size."
0% [██████████████████████████████] 100% | ETA: 00:00:00
Total time elapsed: 00:00:51


Finished Epoch 1 || Run Time:   21.9 | Load Time:   29.7 || F1:  65.69 | Prec:  59.35 | Rec:  73.56 || Ex/s: 333.58

===>  EVAL Epoch 1


0% [██████████████████████████████] 100% | ETA: 00:00:00
Total time elapsed: 00:00:11


...

0% [██████████████████████████████] 100% | ETA: 00:00:00
Total time elapsed: 00:00:47


Finished Epoch 15 || Run Time:   20.2 | Load Time:   27.8 || F1:  99.86 | Prec:  99.84 | Rec:  99.88 || Ex/s: 358.93

===>  EVAL Epoch 15


0% [██████████████████████████████] 100% | ETA: 00:00:00
Total time elapsed: 00:00:11


Finished Epoch 15 || Run Time:    3.0 | Load Time:    8.4 || F1:  83.90 | Prec:  82.17 | Rec:  85.70 || Ex/s: 502.43

---------------------

Loading best model...
Training done.


tensor(84.7081, device='cuda:0')

In [14]:
# Compute F1 on test set
model.run_eval(test)

===>  EVAL Epoch 8


/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:949: UserWarning: Using non-full backward hooks on a Module that does not take as input a single Tensor or a tuple of Tensors is deprecated and will be removed in future versions. This hook will be missing some of the grad_input. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using non-full backward hooks on a Module that does not take as input a "
/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:939: UserWarning: Using non-full backward hooks on a Module that does not return a single Tensor or a tuple of Tensors is deprecated and will be removed in future versions. This hook will be missing some of the grad_output. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using non-full backward hooks on a Module that does not return a "
/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:974: UserWarning: Using a non-f

Finished Epoch 8 || Run Time:    3.8 | Load Time:   10.2 || F1:  82.57 | Prec:  83.00 | Rec:  82.15 || Ex/s: 411.44



tensor(82.5740, device='cuda:0')

### SIF

#### STRUCTURED

##### Amazon-Google

In [25]:
train, validation, test = dm.data.process(
    path='/content/IC/datasesErros/CMedium_30_3_5/Structured/Amazon-Google/',
    train='joined_train.csv',
    validation='joined_valid.csv',
    test='joined_test.csv')

In [26]:
model = dm.MatchingModel(attr_summarizer='sif')

In [27]:
model.run_train(
    train,
    validation,
    epochs=15,
    batch_size=32,
    best_save_path='sif_model.pth',
    pos_neg_ratio=3)

* Number of trainable parameters: 542402
===>  TRAIN Epoch 1


/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:974: UserWarning: Using a non-full backward hook when the forward contains multiple autograd Nodes is deprecated and will be removed in future versions. This hook will be missing some grad_input. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using a non-full backward hook when the forward contains multiple autograd Nodes "
/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:2742: UserWarning: reduction: 'mean' divides the total loss by both the batch size and the support size.'batchmean' divides only by the batch size, and aligns with the KL div math definition.'mean' will be changed to behave the same as 'batchmean' in the next major release.
  "reduction: 'mean' divides the total loss by both the batch size and the support size."
0% [██████████████████████████████] 100% | ETA: 00:00:00
Total time elapsed: 00:00:07


Finished Epoch 1 || Run Time:    3.0 | Load Time:    4.6 || F1:  15.19 | Prec:  38.82 | Rec:   9.44 || Ex/s: 904.88

===>  EVAL Epoch 1


0% [██████████████] 100% | ETA: 00:00:00
Total time elapsed: 00:00:02


Finished Epoch 1 || Run Time:    0.7 | Load Time:    1.6 || F1:  30.11 | Prec:  31.75 | Rec:  28.63 || Ex/s: 1018.73

* Best F1: tensor(30.1124, device='cuda:0')
Saving best model...
Done.
---------------------

===>  TRAIN Epoch 2


...

0% [██████████████████████████████] 100% | ETA: 00:00:00
Total time elapsed: 00:00:07


Finished Epoch 15 || Run Time:    3.2 | Load Time:    4.8 || F1:  92.41 | Prec:  88.48 | Rec:  96.71 || Ex/s: 867.19

===>  EVAL Epoch 15


0% [██████████████] 100% | ETA: 00:00:00
Total time elapsed: 00:00:01


Finished Epoch 15 || Run Time:    0.6 | Load Time:    1.4 || F1:  38.51 | Prec:  37.35 | Rec:  39.74 || Ex/s: 1128.13

---------------------

Loading best model...
Training done.


tensor(41.9014, device='cuda:0')

In [28]:
# Compute F1 on test set
model.run_eval(test)

===>  EVAL Epoch 3


/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:974: UserWarning: Using a non-full backward hook when the forward contains multiple autograd Nodes is deprecated and will be removed in future versions. This hook will be missing some grad_input. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using a non-full backward hook when the forward contains multiple autograd Nodes "
0% [██████████████] 100% | ETA: 00:00:00
Total time elapsed: 00:00:02


Finished Epoch 3 || Run Time:    0.7 | Load Time:    1.6 || F1:  43.83 | Prec:  38.51 | Rec:  50.85 || Ex/s: 1007.55



tensor(43.8306, device='cuda:0')

##### Beer

In [29]:
train, validation, test = dm.data.process(
    path='/content/IC/datasesErros/CMedium_30_3_5/Structured/Beer/',
    train='joined_train.csv',
    validation='joined_valid.csv',
    test='joined_test.csv')

In [30]:
model = dm.MatchingModel(attr_summarizer='sif')

In [31]:
model.run_train(
    train,
    validation,
    epochs=15,
    batch_size=32,
    best_save_path='sif_model.pth',
    pos_neg_ratio=3)

* Number of trainable parameters: 542402
===>  TRAIN Epoch 1


/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:974: UserWarning: Using a non-full backward hook when the forward contains multiple autograd Nodes is deprecated and will be removed in future versions. This hook will be missing some grad_input. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using a non-full backward hook when the forward contains multiple autograd Nodes "
/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:2742: UserWarning: reduction: 'mean' divides the total loss by both the batch size and the support size.'batchmean' divides only by the batch size, and aligns with the KL div math definition.'mean' will be changed to behave the same as 'batchmean' in the next major release.
  "reduction: 'mean' divides the total loss by both the batch size and the support size."
0% [█] 100% | ETA: 00:00:00
Total time elapsed: 00:00:00


Finished Epoch 1 || Run Time:    0.2 | Load Time:    0.3 || F1:   0.00 | Prec:   0.00 | Rec:   0.00 || Ex/s: 609.78

===>  EVAL Epoch 1
Finished Epoch 1 || Run Time:    0.0 | Load Time:    0.1 || F1:   0.00 | Prec:   0.00 | Rec:   0.00 || Ex/s: 700.68

* Best F1: tensor(0., device='cuda:0')
Saving best model...
Done.
---------------------

===>  TRAIN Epoch 2


0% [█] 100% | ETA: 00:00:00
Total time elapsed: 00:00:00


Finished Epoch 2 || Run Time:    0.2 | Load Time:    0.3 || F1:   0.00 | Prec:   0.00 | Rec:   0.00 || Ex/s: 624.18

===>  EVAL Epoch 2
Finished Epoch 2 || Run Time:    0.0 | Load Time:    0.1 || F1:   0.00 | Prec:   0.00 | Rec:   0.00 || Ex/s: 681.78

---------------------

===>  TRAIN Epoch 3


...

0% [█] 100% | ETA: 00:00:00
Total time elapsed: 00:00:00


Finished Epoch 14 || Run Time:    0.2 | Load Time:    0.3 || F1:  46.43 | Prec:  81.25 | Rec:  32.50 || Ex/s: 618.69

===>  EVAL Epoch 14
Finished Epoch 14 || Run Time:    0.0 | Load Time:    0.1 || F1:  40.00 | Prec:  66.67 | Rec:  28.57 || Ex/s: 742.51

---------------------

===>  TRAIN Epoch 15


0% [█] 100% | ETA: 00:00:00
Total time elapsed: 00:00:00


Finished Epoch 15 || Run Time:    0.2 | Load Time:    0.3 || F1:  46.43 | Prec:  81.25 | Rec:  32.50 || Ex/s: 633.91

===>  EVAL Epoch 15
Finished Epoch 15 || Run Time:    0.0 | Load Time:    0.1 || F1:  40.00 | Prec:  66.67 | Rec:  28.57 || Ex/s: 686.97

---------------------

Loading best model...
Training done.


tensor(40., device='cuda:0')

In [32]:
# Compute F1 on test set
model.run_eval(test)

===>  EVAL Epoch 13


/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:974: UserWarning: Using a non-full backward hook when the forward contains multiple autograd Nodes is deprecated and will be removed in future versions. This hook will be missing some grad_input. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using a non-full backward hook when the forward contains multiple autograd Nodes "


Finished Epoch 13 || Run Time:    0.0 | Load Time:    0.1 || F1:  20.00 | Prec:  33.33 | Rec:  14.29 || Ex/s: 699.93



tensor(20., device='cuda:0')

##### DBLP-ACM

In [33]:
train, validation, test = dm.data.process(
    path='/content/IC/datasesErros/CMedium_30_3_5/Structured/DBLP-ACM/',
    train='joined_train.csv',
    validation='joined_valid.csv',
    test='joined_test.csv')

In [34]:
model = dm.MatchingModel(attr_summarizer='sif')

In [35]:
model.run_train(
    train,
    validation,
    epochs=15,
    batch_size=32,
    best_save_path='sif_model.pth',
    pos_neg_ratio=3)

* Number of trainable parameters: 542402
===>  TRAIN Epoch 1


/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:974: UserWarning: Using a non-full backward hook when the forward contains multiple autograd Nodes is deprecated and will be removed in future versions. This hook will be missing some grad_input. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using a non-full backward hook when the forward contains multiple autograd Nodes "
/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:2742: UserWarning: reduction: 'mean' divides the total loss by both the batch size and the support size.'batchmean' divides only by the batch size, and aligns with the KL div math definition.'mean' will be changed to behave the same as 'batchmean' in the next major release.
  "reduction: 'mean' divides the total loss by both the batch size and the support size."
0% [██████████████████████████████] 100% | ETA: 00:00:00
Total time elapsed: 00:00:15


Finished Epoch 1 || Run Time:    3.9 | Load Time:   11.4 || F1:  74.76 | Prec:  77.82 | Rec:  71.92 || Ex/s: 483.15

===>  EVAL Epoch 1


0% [███████████████] 100% | ETA: 00:00:00
Total time elapsed: 00:00:04


Finished Epoch 1 || Run Time:    0.8 | Load Time:    3.6 || F1:  92.39 | Prec:  90.50 | Rec:  94.37 || Ex/s: 559.46

* Best F1: tensor(92.3925, device='cuda:0')
Saving best model...
Done.
---------------------

===>  TRAIN Epoch 2


...

0% [██████████████████████████████] 100% | ETA: 00:00:00
Total time elapsed: 00:00:15


Finished Epoch 15 || Run Time:    4.0 | Load Time:   11.5 || F1:  99.55 | Prec:  99.11 | Rec: 100.00 || Ex/s: 477.65

===>  EVAL Epoch 15


0% [███████████████] 100% | ETA: 00:00:00
Total time elapsed: 00:00:04


Finished Epoch 15 || Run Time:    0.8 | Load Time:    3.6 || F1:  95.05 | Prec:  92.90 | Rec:  97.30 || Ex/s: 553.12

* Best F1: tensor(95.0495, device='cuda:0')
Saving best model...
Done.
---------------------

Loading best model...
Training done.


tensor(95.0495, device='cuda:0')

In [36]:
# Compute F1 on test set
model.run_eval(test)

===>  EVAL Epoch 15


/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:974: UserWarning: Using a non-full backward hook when the forward contains multiple autograd Nodes is deprecated and will be removed in future versions. This hook will be missing some grad_input. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using a non-full backward hook when the forward contains multiple autograd Nodes "
0% [███████████████] 100% | ETA: 00:00:00
Total time elapsed: 00:00:04


Finished Epoch 15 || Run Time:    0.8 | Load Time:    3.6 || F1:  95.04 | Prec:  93.09 | Rec:  97.07 || Ex/s: 564.28



tensor(95.0386, device='cuda:0')

##### DBLP-GoogleScholar

In [37]:
train, validation, test = dm.data.process(
    path='/content/IC/datasesErros/CMedium_30_3_5/Structured/DBLP-GoogleScholar/',
    train='joined_train.csv',
    validation='joined_valid.csv',
    test='joined_test.csv')

In [38]:
model = dm.MatchingModel(attr_summarizer='sif')

In [39]:
model.run_train(
    train,
    validation,
    epochs=15,
    batch_size=32,
    best_save_path='sif_model.pth',
    pos_neg_ratio=3)

* Number of trainable parameters: 542402
===>  TRAIN Epoch 1


/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:974: UserWarning: Using a non-full backward hook when the forward contains multiple autograd Nodes is deprecated and will be removed in future versions. This hook will be missing some grad_input. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using a non-full backward hook when the forward contains multiple autograd Nodes "
/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:2742: UserWarning: reduction: 'mean' divides the total loss by both the batch size and the support size.'batchmean' divides only by the batch size, and aligns with the KL div math definition.'mean' will be changed to behave the same as 'batchmean' in the next major release.
  "reduction: 'mean' divides the total loss by both the batch size and the support size."
0% [██████████████████████████████] 100% | ETA: 00:00:00
Total time elapsed: 00:00:32


Finished Epoch 1 || Run Time:    9.0 | Load Time:   24.0 || F1:  63.53 | Prec:  60.49 | Rec:  66.88 || Ex/s: 521.19

===>  EVAL Epoch 1


0% [██████████████████████████████] 100% | ETA: 00:00:00
Total time elapsed: 00:00:09


Finished Epoch 1 || Run Time:    1.9 | Load Time:    7.3 || F1:  76.08 | Prec:  69.96 | Rec:  83.36 || Ex/s: 625.69

* Best F1: tensor(76.0768, device='cuda:0')
Saving best model...
Done.
---------------------

===>  TRAIN Epoch 2


...

0% [██████████████████████████████] 100% | ETA: 00:00:00
Total time elapsed: 00:00:32


Finished Epoch 15 || Run Time:    8.8 | Load Time:   23.6 || F1:  99.19 | Prec:  98.82 | Rec:  99.56 || Ex/s: 530.78

===>  EVAL Epoch 15


0% [██████████████████████████████] 100% | ETA: 00:00:00
Total time elapsed: 00:00:09


Finished Epoch 15 || Run Time:    1.9 | Load Time:    7.2 || F1:  80.04 | Prec:  83.10 | Rec:  77.20 || Ex/s: 631.50

---------------------

Loading best model...
Training done.


tensor(80.9062, device='cuda:0')

In [40]:
# Compute F1 on test set
model.run_eval(test)

===>  EVAL Epoch 7


/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:974: UserWarning: Using a non-full backward hook when the forward contains multiple autograd Nodes is deprecated and will be removed in future versions. This hook will be missing some grad_input. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using a non-full backward hook when the forward contains multiple autograd Nodes "
0% [██████████████████████████████] 100% | ETA: 00:00:00
Total time elapsed: 00:00:09


Finished Epoch 7 || Run Time:    1.9 | Load Time:    7.3 || F1:  80.71 | Prec:  78.98 | Rec:  82.52 || Ex/s: 622.62



tensor(80.7130, device='cuda:0')

##### Walmart-Amazon

In [41]:
train, validation, test = dm.data.process(
    path='/content/IC/datasesErros/CMedium_30_3_5/Structured/Walmart-Amazon/',
    train='joined_train.csv',
    validation='joined_valid.csv',
    test='joined_test.csv')

In [42]:
model = dm.MatchingModel(attr_summarizer='sif')

In [43]:
model.run_train(
    train,
    validation,
    epochs=15,
    batch_size=32,
    best_save_path='sif_model.pth',
    pos_neg_ratio=3)

* Number of trainable parameters: 542402
===>  TRAIN Epoch 1


/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:974: UserWarning: Using a non-full backward hook when the forward contains multiple autograd Nodes is deprecated and will be removed in future versions. This hook will be missing some grad_input. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using a non-full backward hook when the forward contains multiple autograd Nodes "
/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:2742: UserWarning: reduction: 'mean' divides the total loss by both the batch size and the support size.'batchmean' divides only by the batch size, and aligns with the KL div math definition.'mean' will be changed to behave the same as 'batchmean' in the next major release.
  "reduction: 'mean' divides the total loss by both the batch size and the support size."
0% [██████████████████████████████] 100% | ETA: 00:00:00
Total time elapsed: 00:00:10


Finished Epoch 1 || Run Time:    3.8 | Load Time:    6.5 || F1:  23.11 | Prec:  44.33 | Rec:  15.62 || Ex/s: 595.05

===>  EVAL Epoch 1


0% [█████████████] 100% | ETA: 00:00:00
Total time elapsed: 00:00:02


Finished Epoch 1 || Run Time:    0.8 | Load Time:    2.0 || F1:  47.71 | Prec:  42.80 | Rec:  53.89 || Ex/s: 724.19

* Best F1: tensor(47.7064, device='cuda:0')
Saving best model...
Done.
---------------------

===>  TRAIN Epoch 2


...

0% [██████████████████████████████] 100% | ETA: 00:00:00
Total time elapsed: 00:00:10


Finished Epoch 15 || Run Time:    3.8 | Load Time:    6.4 || F1:  99.22 | Prec:  99.30 | Rec:  99.13 || Ex/s: 601.73

===>  EVAL Epoch 15


0% [█████████████] 100% | ETA: 00:00:00
Total time elapsed: 00:00:02


Finished Epoch 15 || Run Time:    0.8 | Load Time:    2.0 || F1:  57.61 | Prec:  60.57 | Rec:  54.92 || Ex/s: 717.60

* Best F1: tensor(57.6087, device='cuda:0')
Saving best model...
Done.
---------------------

Loading best model...
Training done.


tensor(57.6087, device='cuda:0')

In [44]:
# Compute F1 on test set
model.run_eval(test)

===>  EVAL Epoch 15


/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:974: UserWarning: Using a non-full backward hook when the forward contains multiple autograd Nodes is deprecated and will be removed in future versions. This hook will be missing some grad_input. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using a non-full backward hook when the forward contains multiple autograd Nodes "
0% [█████████████] 100% | ETA: 00:00:00
Total time elapsed: 00:00:02


Finished Epoch 15 || Run Time:    0.8 | Load Time:    2.0 || F1:  54.49 | Prec:  59.51 | Rec:  50.26 || Ex/s: 713.16



tensor(54.4944, device='cuda:0')

#### TEXTUAL

##### Abt-Buy

In [45]:
train, validation, test = dm.data.process(
    path='/content/IC/datasesErros/CMedium_30_3_5/Textual/Abt-Buy/',
    train='joined_train.csv',
    validation='joined_valid.csv',
    test='joined_test.csv')

In [46]:
model = dm.MatchingModel(attr_summarizer='sif')

In [47]:
model.run_train(
    train,
    validation,
    epochs=15,
    batch_size=32,
    best_save_path='sif_model.pth',
    pos_neg_ratio=3)

* Number of trainable parameters: 542402
===>  TRAIN Epoch 1


/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:974: UserWarning: Using a non-full backward hook when the forward contains multiple autograd Nodes is deprecated and will be removed in future versions. This hook will be missing some grad_input. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using a non-full backward hook when the forward contains multiple autograd Nodes "
/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:2742: UserWarning: reduction: 'mean' divides the total loss by both the batch size and the support size.'batchmean' divides only by the batch size, and aligns with the KL div math definition.'mean' will be changed to behave the same as 'batchmean' in the next major release.
  "reduction: 'mean' divides the total loss by both the batch size and the support size."
0% [██████████████████████████████] 100% | ETA: 00:00:00
Total time elapsed: 00:00:12


Finished Epoch 1 || Run Time:    2.5 | Load Time:   10.3 || F1:   0.62 | Prec:   7.69 | Rec:   0.32 || Ex/s: 446.88

===>  EVAL Epoch 1


0% [████████████] 100% | ETA: 00:00:00
Total time elapsed: 00:00:03


Finished Epoch 1 || Run Time:    0.5 | Load Time:    3.1 || F1:   0.00 | Prec:   0.00 | Rec:   0.00 || Ex/s: 532.82

* Best F1: tensor(0., device='cuda:0')
Saving best model...
Done.
---------------------

===>  TRAIN Epoch 2


...

0% [██████████████████████████████] 100% | ETA: 00:00:00
Total time elapsed: 00:00:12


Finished Epoch 15 || Run Time:    2.6 | Load Time:   10.3 || F1:  95.91 | Prec:  92.99 | Rec:  99.03 || Ex/s: 448.20

===>  EVAL Epoch 15


0% [████████████] 100% | ETA: 00:00:00
Total time elapsed: 00:00:03


Finished Epoch 15 || Run Time:    0.5 | Load Time:    3.0 || F1:  16.52 | Prec:  21.05 | Rec:  13.59 || Ex/s: 541.64

---------------------

Loading best model...
Training done.


tensor(26.7490, device='cuda:0')

In [48]:
# Compute F1 on test set
model.run_eval(test)

===>  EVAL Epoch 4


/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:974: UserWarning: Using a non-full backward hook when the forward contains multiple autograd Nodes is deprecated and will be removed in future versions. This hook will be missing some grad_input. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using a non-full backward hook when the forward contains multiple autograd Nodes "
0% [████████████] 100% | ETA: 00:00:00
Total time elapsed: 00:00:03


Finished Epoch 4 || Run Time:    0.5 | Load Time:    3.1 || F1:  24.51 | Prec:  20.67 | Rec:  30.10 || Ex/s: 533.70



tensor(24.5059, device='cuda:0')

#### DIRTY

##### DBLP-ACM

In [49]:
train, validation, test = dm.data.process(
    path='/content/IC/datasesErros/CMedium_30_3_5/Dirty/DBLP-ACM/',
    train='joined_train.csv',
    validation='joined_valid.csv',
    test='joined_test.csv')

In [50]:
model = dm.MatchingModel(attr_summarizer='sif')

In [51]:
model.run_train(
    train,
    validation,
    epochs=15,
    batch_size=32,
    best_save_path='sif_model.pth',
    pos_neg_ratio=3)

* Number of trainable parameters: 542402
===>  TRAIN Epoch 1


/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:974: UserWarning: Using a non-full backward hook when the forward contains multiple autograd Nodes is deprecated and will be removed in future versions. This hook will be missing some grad_input. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using a non-full backward hook when the forward contains multiple autograd Nodes "
/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:2742: UserWarning: reduction: 'mean' divides the total loss by both the batch size and the support size.'batchmean' divides only by the batch size, and aligns with the KL div math definition.'mean' will be changed to behave the same as 'batchmean' in the next major release.
  "reduction: 'mean' divides the total loss by both the batch size and the support size."
0% [██████████████████████████████] 100% | ETA: 00:00:00
Total time elapsed: 00:00:17


Finished Epoch 1 || Run Time:    3.9 | Load Time:   14.0 || F1:  44.10 | Prec:  55.73 | Rec:  36.49 || Ex/s: 414.18

===>  EVAL Epoch 1


0% [███████████████] 100% | ETA: 00:00:00
Total time elapsed: 00:00:05


Finished Epoch 1 || Run Time:    0.9 | Load Time:    4.8 || F1:  62.63 | Prec:  71.26 | Rec:  55.86 || Ex/s: 437.28

* Best F1: tensor(62.6263, device='cuda:0')
Saving best model...
Done.
---------------------

===>  TRAIN Epoch 2


...

0% [██████████████████████████████] 100% | ETA: 00:00:00
Total time elapsed: 00:00:17


Finished Epoch 15 || Run Time:    3.9 | Load Time:   14.0 || F1:  98.44 | Prec:  97.36 | Rec:  99.55 || Ex/s: 414.45

===>  EVAL Epoch 15


0% [███████████████] 100% | ETA: 00:00:00
Total time elapsed: 00:00:05


Finished Epoch 15 || Run Time:    0.8 | Load Time:    4.4 || F1:  72.13 | Prec:  72.87 | Rec:  71.40 || Ex/s: 471.31

---------------------

Loading best model...
Training done.


tensor(72.9730, device='cuda:0')

In [52]:
# Compute F1 on test set
model.run_eval(test)

===>  EVAL Epoch 5


/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:974: UserWarning: Using a non-full backward hook when the forward contains multiple autograd Nodes is deprecated and will be removed in future versions. This hook will be missing some grad_input. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using a non-full backward hook when the forward contains multiple autograd Nodes "
0% [███████████████] 100% | ETA: 00:00:00
Total time elapsed: 00:00:05


Finished Epoch 5 || Run Time:    0.8 | Load Time:    4.4 || F1:  73.77 | Prec:  68.81 | Rec:  79.50 || Ex/s: 476.16



tensor(73.7722, device='cuda:0')

##### DBLP-GoogleScholar

In [53]:
train, validation, test = dm.data.process(
    path='/content/IC/datasesErros/CMedium_30_3_5/Dirty/DBLP-GoogleScholar//',
    train='joined_train.csv',
    validation='joined_valid.csv',
    test='joined_test.csv')

In [54]:
model = dm.MatchingModel(attr_summarizer='sif')

In [55]:
model.run_train(
    train,
    validation,
    epochs=15,
    batch_size=32,
    best_save_path='sif_model.pth',
    pos_neg_ratio=3)

* Number of trainable parameters: 542402
===>  TRAIN Epoch 1


/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:974: UserWarning: Using a non-full backward hook when the forward contains multiple autograd Nodes is deprecated and will be removed in future versions. This hook will be missing some grad_input. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using a non-full backward hook when the forward contains multiple autograd Nodes "
/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:2742: UserWarning: reduction: 'mean' divides the total loss by both the batch size and the support size.'batchmean' divides only by the batch size, and aligns with the KL div math definition.'mean' will be changed to behave the same as 'batchmean' in the next major release.
  "reduction: 'mean' divides the total loss by both the batch size and the support size."
0% [██████████████████████████████] 100% | ETA: 00:00:00
Total time elapsed: 00:00:36


Finished Epoch 1 || Run Time:    8.9 | Load Time:   28.1 || F1:  48.74 | Prec:  48.75 | Rec:  48.74 || Ex/s: 465.87

===>  EVAL Epoch 1


0% [██████████████████████████████] 100% | ETA: 00:00:00
Total time elapsed: 00:00:10


Finished Epoch 1 || Run Time:    1.9 | Load Time:    8.6 || F1:  61.91 | Prec:  51.76 | Rec:  77.01 || Ex/s: 545.88

* Best F1: tensor(61.9083, device='cuda:0')
Saving best model...
Done.
---------------------

===>  TRAIN Epoch 2


...

0% [██████████████████████████████] 100% | ETA: 00:00:00
Total time elapsed: 00:00:37


Finished Epoch 15 || Run Time:    9.0 | Load Time:   28.4 || F1:  98.67 | Prec:  99.09 | Rec:  98.25 || Ex/s: 461.30

===>  EVAL Epoch 15


0% [██████████████████████████████] 100% | ETA: 00:00:00
Total time elapsed: 00:00:10


Finished Epoch 15 || Run Time:    1.9 | Load Time:    8.6 || F1:  71.70 | Prec:  72.88 | Rec:  70.56 || Ex/s: 545.33

---------------------

Loading best model...
Training done.


tensor(72.1408, device='cuda:0')

In [56]:
# Compute F1 on test set
model.run_eval(test)

===>  EVAL Epoch 8


/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:974: UserWarning: Using a non-full backward hook when the forward contains multiple autograd Nodes is deprecated and will be removed in future versions. This hook will be missing some grad_input. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using a non-full backward hook when the forward contains multiple autograd Nodes "
0% [██████████████████████████████] 100% | ETA: 00:00:00
Total time elapsed: 00:00:10


Finished Epoch 8 || Run Time:    1.9 | Load Time:    8.8 || F1:  71.46 | Prec:  73.40 | Rec:  69.63 || Ex/s: 535.44



tensor(71.4628, device='cuda:0')

### ATTENTION

#### STRUCTURED

##### Amazon-Google

In [57]:
train, validation, test = dm.data.process(
    path='/content/IC/datasesErros/CMedium_30_3_5/Structured/Amazon-Google/',
    train='joined_train.csv',
    validation='joined_valid.csv',
    test='joined_test.csv')

In [58]:
model = dm.MatchingModel(attr_summarizer='attention')

In [59]:
model.run_train(
    train,
    validation,
    epochs=15,
    batch_size=32,
    best_save_path='attention_model.pth',
    pos_neg_ratio=3)

* Number of trainable parameters: 3429602
===>  TRAIN Epoch 1


/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:974: UserWarning: Using a non-full backward hook when the forward contains multiple autograd Nodes is deprecated and will be removed in future versions. This hook will be missing some grad_input. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using a non-full backward hook when the forward contains multiple autograd Nodes "
/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:949: UserWarning: Using non-full backward hooks on a Module that does not take as input a single Tensor or a tuple of Tensors is deprecated and will be removed in future versions. This hook will be missing some of the grad_input. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using non-full backward hooks on a Module that does not take as input a "
/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:939: UserWarning: Using non-full backward h

Finished Epoch 1 || Run Time:   10.8 | Load Time:    4.5 || F1:  25.35 | Prec:  32.58 | Rec:  20.74 || Ex/s: 450.42

===>  EVAL Epoch 1


0% [██████████████] 100% | ETA: 00:00:00
Total time elapsed: 00:00:03


Finished Epoch 1 || Run Time:    2.1 | Load Time:    1.4 || F1:  38.79 | Prec:  32.03 | Rec:  49.15 || Ex/s: 658.87

* Best F1: tensor(38.7858, device='cuda:0')
Saving best model...
Done.
---------------------

===>  TRAIN Epoch 2


...

0% [██████████████████████████████] 100% | ETA: 00:00:00
Total time elapsed: 00:00:15


Finished Epoch 15 || Run Time:   10.8 | Load Time:    4.5 || F1:  92.37 | Prec:  88.17 | Rec:  97.00 || Ex/s: 448.54

===>  EVAL Epoch 15


0% [██████████████] 100% | ETA: 00:00:00
Total time elapsed: 00:00:03


Finished Epoch 15 || Run Time:    2.1 | Load Time:    1.4 || F1:  47.09 | Prec:  49.53 | Rec:  44.87 || Ex/s: 661.19

---------------------

Loading best model...
Training done.


tensor(49.1879, device='cuda:0')

In [60]:
# Compute F1 on test set
model.run_eval(test)

===>  EVAL Epoch 9


/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:974: UserWarning: Using a non-full backward hook when the forward contains multiple autograd Nodes is deprecated and will be removed in future versions. This hook will be missing some grad_input. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using a non-full backward hook when the forward contains multiple autograd Nodes "
/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:949: UserWarning: Using non-full backward hooks on a Module that does not take as input a single Tensor or a tuple of Tensors is deprecated and will be removed in future versions. This hook will be missing some of the grad_input. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using non-full backward hooks on a Module that does not take as input a "
/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:939: UserWarning: Using non-full backward h

Finished Epoch 9 || Run Time:    2.1 | Load Time:    1.4 || F1:  46.47 | Prec:  49.76 | Rec:  43.59 || Ex/s: 655.36



tensor(46.4692, device='cuda:0')

##### Beer

In [61]:
train, validation, test = dm.data.process(
    path='/content/IC/datasesErros/CMedium_30_3_5/Structured/Beer/',
    train='joined_train.csv',
    validation='joined_valid.csv',
    test='joined_test.csv')

In [62]:
model = dm.MatchingModel(attr_summarizer='attention')

In [63]:
model.run_train(
    train,
    validation,
    epochs=15,
    batch_size=32,
    best_save_path='attention_model.pth',
    pos_neg_ratio=3)

/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:974: UserWarning: Using a non-full backward hook when the forward contains multiple autograd Nodes is deprecated and will be removed in future versions. This hook will be missing some grad_input. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using a non-full backward hook when the forward contains multiple autograd Nodes "
/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:949: UserWarning: Using non-full backward hooks on a Module that does not take as input a single Tensor or a tuple of Tensors is deprecated and will be removed in future versions. This hook will be missing some of the grad_input. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using non-full backward hooks on a Module that does not take as input a "


* Number of trainable parameters: 4332002
===>  TRAIN Epoch 1


/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:939: UserWarning: Using non-full backward hooks on a Module that does not return a single Tensor or a tuple of Tensors is deprecated and will be removed in future versions. This hook will be missing some of the grad_output. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using non-full backward hooks on a Module that does not return a "
/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:2742: UserWarning: reduction: 'mean' divides the total loss by both the batch size and the support size.'batchmean' divides only by the batch size, and aligns with the KL div math definition.'mean' will be changed to behave the same as 'batchmean' in the next major release.
  "reduction: 'mean' divides the total loss by both the batch size and the support size."
0% [█] 100% | ETA: 00:00:00
Total time elapsed: 00:00:00


Finished Epoch 1 || Run Time:    0.6 | Load Time:    0.3 || F1:  20.65 | Prec:  13.91 | Rec:  40.00 || Ex/s: 295.93

===>  EVAL Epoch 1
Finished Epoch 1 || Run Time:    0.1 | Load Time:    0.1 || F1:  28.00 | Prec:  16.28 | Rec: 100.00 || Ex/s: 422.05

* Best F1: tensor(28., device='cuda:0')
Saving best model...
Done.
---------------------

===>  TRAIN Epoch 2


0% [█] 100% | ETA: 00:00:00
Total time elapsed: 00:00:00


...

0% [█] 100% | ETA: 00:00:00
Total time elapsed: 00:00:00


Finished Epoch 14 || Run Time:    0.6 | Load Time:    0.3 || F1:  86.05 | Prec:  80.43 | Rec:  92.50 || Ex/s: 302.47

===>  EVAL Epoch 14
Finished Epoch 14 || Run Time:    0.1 | Load Time:    0.1 || F1:  43.75 | Prec:  38.89 | Rec:  50.00 || Ex/s: 439.73

---------------------

===>  TRAIN Epoch 15


0% [█] 100% | ETA: 00:00:00
Total time elapsed: 00:00:00


Finished Epoch 15 || Run Time:    0.6 | Load Time:    0.3 || F1:  86.05 | Prec:  80.43 | Rec:  92.50 || Ex/s: 307.38

===>  EVAL Epoch 15
Finished Epoch 15 || Run Time:    0.1 | Load Time:    0.1 || F1:  43.75 | Prec:  38.89 | Rec:  50.00 || Ex/s: 443.48

---------------------

Loading best model...
Training done.


tensor(43.7500, device='cuda:0')

In [64]:
# Compute F1 on test set
model.run_eval(test)

===>  EVAL Epoch 13


/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:974: UserWarning: Using a non-full backward hook when the forward contains multiple autograd Nodes is deprecated and will be removed in future versions. This hook will be missing some grad_input. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using a non-full backward hook when the forward contains multiple autograd Nodes "
/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:949: UserWarning: Using non-full backward hooks on a Module that does not take as input a single Tensor or a tuple of Tensors is deprecated and will be removed in future versions. This hook will be missing some of the grad_input. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using non-full backward hooks on a Module that does not take as input a "
/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:939: UserWarning: Using non-full backward h

Finished Epoch 13 || Run Time:    0.1 | Load Time:    0.1 || F1:  47.06 | Prec:  40.00 | Rec:  57.14 || Ex/s: 414.78



tensor(47.0588, device='cuda:0')

##### DBLP-ACM

In [65]:
train, validation, test = dm.data.process(
    path='/content/IC/datasesErros/CMedium_30_3_5/Structured/DBLP-ACM/',
    train='joined_train.csv',
    validation='joined_valid.csv',
    test='joined_test.csv')

In [66]:
model = dm.MatchingModel(attr_summarizer='attention')

In [67]:
model.run_train(
    train,
    validation,
    epochs=15,
    batch_size=32,
    best_save_path='attention_model.pth',
    pos_neg_ratio=3)

* Number of trainable parameters: 4332002
===>  TRAIN Epoch 1


/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:974: UserWarning: Using a non-full backward hook when the forward contains multiple autograd Nodes is deprecated and will be removed in future versions. This hook will be missing some grad_input. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using a non-full backward hook when the forward contains multiple autograd Nodes "
/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:949: UserWarning: Using non-full backward hooks on a Module that does not take as input a single Tensor or a tuple of Tensors is deprecated and will be removed in future versions. This hook will be missing some of the grad_input. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using non-full backward hooks on a Module that does not take as input a "
/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:939: UserWarning: Using non-full backward h

Finished Epoch 1 || Run Time:   15.9 | Load Time:   11.4 || F1:  84.37 | Prec:  78.36 | Rec:  91.37 || Ex/s: 271.82

===>  EVAL Epoch 1


0% [███████████████] 100% | ETA: 00:00:00
Total time elapsed: 00:00:06


Finished Epoch 1 || Run Time:    3.4 | Load Time:    3.8 || F1:  96.50 | Prec:  93.83 | Rec:  99.32 || Ex/s: 342.90

* Best F1: tensor(96.4989, device='cuda:0')
Saving best model...
Done.
---------------------

===>  TRAIN Epoch 2


...

0% [██████████████████████████████] 100% | ETA: 00:00:00
Total time elapsed: 00:00:27


Finished Epoch 15 || Run Time:   16.0 | Load Time:   11.4 || F1:  99.40 | Prec:  98.81 | Rec: 100.00 || Ex/s: 270.49

===>  EVAL Epoch 15


0% [███████████████] 100% | ETA: 00:00:00
Total time elapsed: 00:00:06


Finished Epoch 15 || Run Time:    3.1 | Load Time:    3.6 || F1:  97.76 | Prec:  97.11 | Rec:  98.42 || Ex/s: 368.00

---------------------

Loading best model...
Training done.


tensor(97.8771, device='cuda:0')

In [68]:
# Compute F1 on test set
model.run_eval(test)

===>  EVAL Epoch 12


/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:974: UserWarning: Using a non-full backward hook when the forward contains multiple autograd Nodes is deprecated and will be removed in future versions. This hook will be missing some grad_input. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using a non-full backward hook when the forward contains multiple autograd Nodes "
/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:949: UserWarning: Using non-full backward hooks on a Module that does not take as input a single Tensor or a tuple of Tensors is deprecated and will be removed in future versions. This hook will be missing some of the grad_input. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using non-full backward hooks on a Module that does not take as input a "
/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:939: UserWarning: Using non-full backward h

Finished Epoch 12 || Run Time:    3.2 | Load Time:    3.6 || F1:  97.32 | Prec:  96.67 | Rec:  97.97 || Ex/s: 367.05



tensor(97.3154, device='cuda:0')

##### DBLP-GoogleScholar

In [69]:
train, validation, test = dm.data.process(
    path='/content/IC/datasesErros/CMedium_30_3_5/Structured/DBLP-GoogleScholar/',
    train='joined_train.csv',
    validation='joined_valid.csv',
    test='joined_test.csv')

In [70]:
model = dm.MatchingModel(attr_summarizer='attention')

In [71]:
model.run_train(
    train,
    validation,
    epochs=15,
    batch_size=32,
    best_save_path='attention_model.pth',
    pos_neg_ratio=3)

* Number of trainable parameters: 4332002
===>  TRAIN Epoch 1


/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:974: UserWarning: Using a non-full backward hook when the forward contains multiple autograd Nodes is deprecated and will be removed in future versions. This hook will be missing some grad_input. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using a non-full backward hook when the forward contains multiple autograd Nodes "
/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:949: UserWarning: Using non-full backward hooks on a Module that does not take as input a single Tensor or a tuple of Tensors is deprecated and will be removed in future versions. This hook will be missing some of the grad_input. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using non-full backward hooks on a Module that does not take as input a "
/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:939: UserWarning: Using non-full backward h

Finished Epoch 1 || Run Time:   36.3 | Load Time:   23.8 || F1:  79.05 | Prec:  71.59 | Rec:  88.24 || Ex/s: 286.63

===>  EVAL Epoch 1


0% [██████████████████████████████] 100% | ETA: 00:00:00
Total time elapsed: 00:00:14


Finished Epoch 1 || Run Time:    7.1 | Load Time:    7.2 || F1:  79.45 | Prec:  67.10 | Rec:  97.38 || Ex/s: 401.04

* Best F1: tensor(79.4510, device='cuda:0')
Saving best model...
Done.
---------------------

===>  TRAIN Epoch 2


...

0% [██████████████████████████████] 100% | ETA: 00:00:00
Total time elapsed: 00:00:59


Finished Epoch 15 || Run Time:   36.4 | Load Time:   23.7 || F1:  99.33 | Prec:  99.01 | Rec:  99.66 || Ex/s: 286.48

===>  EVAL Epoch 15


0% [██████████████████████████████] 100% | ETA: 00:00:00
Total time elapsed: 00:00:14


Finished Epoch 15 || Run Time:    7.1 | Load Time:    7.3 || F1:  89.23 | Prec:  89.78 | Rec:  88.69 || Ex/s: 400.89

---------------------

Loading best model...
Training done.


tensor(89.8396, device='cuda:0')

In [72]:
# Compute F1 on test set
model.run_eval(test)

===>  EVAL Epoch 3


/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:974: UserWarning: Using a non-full backward hook when the forward contains multiple autograd Nodes is deprecated and will be removed in future versions. This hook will be missing some grad_input. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using a non-full backward hook when the forward contains multiple autograd Nodes "
/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:949: UserWarning: Using non-full backward hooks on a Module that does not take as input a single Tensor or a tuple of Tensors is deprecated and will be removed in future versions. This hook will be missing some of the grad_input. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using non-full backward hooks on a Module that does not take as input a "
/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:939: UserWarning: Using non-full backward h

Finished Epoch 3 || Run Time:    7.1 | Load Time:    7.3 || F1:  89.51 | Prec:  85.34 | Rec:  94.11 || Ex/s: 398.25



tensor(89.5111, device='cuda:0')

##### Walmart-Amazon

In [73]:
train, validation, test = dm.data.process(
    path='/content/IC/datasesErros/CMedium_30_3_5/Structured/Walmart-Amazon/',
    train='joined_train.csv',
    validation='joined_valid.csv',
    test='joined_test.csv')

In [74]:
model = dm.MatchingModel(attr_summarizer='attention')

In [75]:
model.run_train(
    train,
    validation,
    epochs=15,
    batch_size=32,
    best_save_path='attention_model.pth',
    pos_neg_ratio=3)

* Number of trainable parameters: 5234402
===>  TRAIN Epoch 1


/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:974: UserWarning: Using a non-full backward hook when the forward contains multiple autograd Nodes is deprecated and will be removed in future versions. This hook will be missing some grad_input. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using a non-full backward hook when the forward contains multiple autograd Nodes "
/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:949: UserWarning: Using non-full backward hooks on a Module that does not take as input a single Tensor or a tuple of Tensors is deprecated and will be removed in future versions. This hook will be missing some of the grad_input. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using non-full backward hooks on a Module that does not take as input a "
/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:939: UserWarning: Using non-full backward h

Finished Epoch 1 || Run Time:   15.6 | Load Time:    6.5 || F1:  18.03 | Prec:  30.20 | Rec:  12.85 || Ex/s: 277.54

===>  EVAL Epoch 1


0% [█████████████] 100% | ETA: 00:00:00
Total time elapsed: 00:00:05


Finished Epoch 1 || Run Time:    3.0 | Load Time:    2.0 || F1:  42.76 | Prec:  39.47 | Rec:  46.63 || Ex/s: 401.98

* Best F1: tensor(42.7553, device='cuda:0')
Saving best model...
Done.
---------------------

===>  TRAIN Epoch 2


...

0% [██████████████████████████████] 100% | ETA: 00:00:00
Total time elapsed: 00:00:21


Finished Epoch 15 || Run Time:   15.5 | Load Time:    6.4 || F1:  97.85 | Prec:  96.93 | Rec:  98.78 || Ex/s: 279.93

===>  EVAL Epoch 15


0% [█████████████] 100% | ETA: 00:00:00
Total time elapsed: 00:00:05


Finished Epoch 15 || Run Time:    3.3 | Load Time:    2.2 || F1:  48.87 | Prec:  47.55 | Rec:  50.26 || Ex/s: 367.28

---------------------

Loading best model...
Training done.


tensor(51.3317, device='cuda:0')

In [76]:
# Compute F1 on test set
model.run_eval(test)

===>  EVAL Epoch 4


/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:974: UserWarning: Using a non-full backward hook when the forward contains multiple autograd Nodes is deprecated and will be removed in future versions. This hook will be missing some grad_input. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using a non-full backward hook when the forward contains multiple autograd Nodes "
/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:949: UserWarning: Using non-full backward hooks on a Module that does not take as input a single Tensor or a tuple of Tensors is deprecated and will be removed in future versions. This hook will be missing some of the grad_input. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using non-full backward hooks on a Module that does not take as input a "
/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:939: UserWarning: Using non-full backward h

Finished Epoch 4 || Run Time:    3.0 | Load Time:    2.1 || F1:  50.26 | Prec:  50.79 | Rec:  49.74 || Ex/s: 402.77



tensor(50.2618, device='cuda:0')

#### TEXTUAL

##### Abt-Buy

In [77]:
train, validation, test = dm.data.process(
    path='/content/IC/datasesErros/CMedium_30_3_5/Textual/Abt-Buy/',
    train='joined_train.csv',
    validation='joined_valid.csv',
    test='joined_test.csv')

In [78]:
model = dm.MatchingModel(attr_summarizer='attention')

In [79]:
model.run_train(
    train,
    validation,
    epochs=15,
    batch_size=32,
    best_save_path='attention_model.pth',
    pos_neg_ratio=3)

* Number of trainable parameters: 3429602
===>  TRAIN Epoch 1


/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:974: UserWarning: Using a non-full backward hook when the forward contains multiple autograd Nodes is deprecated and will be removed in future versions. This hook will be missing some grad_input. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using a non-full backward hook when the forward contains multiple autograd Nodes "
/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:949: UserWarning: Using non-full backward hooks on a Module that does not take as input a single Tensor or a tuple of Tensors is deprecated and will be removed in future versions. This hook will be missing some of the grad_input. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using non-full backward hooks on a Module that does not take as input a "
/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:939: UserWarning: Using non-full backward h

Finished Epoch 1 || Run Time:   10.8 | Load Time:   10.2 || F1:  17.47 | Prec:  22.08 | Rec:  14.45 || Ex/s: 273.49

===>  EVAL Epoch 1


0% [████████████] 100% | ETA: 00:00:00
Total time elapsed: 00:00:05


Finished Epoch 1 || Run Time:    2.3 | Load Time:    3.0 || F1:   1.89 | Prec:  33.33 | Rec:   0.97 || Ex/s: 356.79

* Best F1: tensor(1.8868, device='cuda:0')
Saving best model...
Done.
---------------------

===>  TRAIN Epoch 2


...

0% [██████████████████████████████] 100% | ETA: 00:00:00
Total time elapsed: 00:00:20


Finished Epoch 15 || Run Time:   10.8 | Load Time:   10.1 || F1:  80.15 | Prec:  72.60 | Rec:  89.45 || Ex/s: 273.58

===>  EVAL Epoch 15


0% [████████████] 100% | ETA: 00:00:00
Total time elapsed: 00:00:05


Finished Epoch 15 || Run Time:    2.0 | Load Time:    3.1 || F1:  40.41 | Prec:  43.33 | Rec:  37.86 || Ex/s: 379.28

---------------------

Loading best model...
Training done.


tensor(42.9245, device='cuda:0')

In [80]:
# Compute F1 on test set
model.run_eval(test)

===>  EVAL Epoch 5


/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:974: UserWarning: Using a non-full backward hook when the forward contains multiple autograd Nodes is deprecated and will be removed in future versions. This hook will be missing some grad_input. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using a non-full backward hook when the forward contains multiple autograd Nodes "
/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:949: UserWarning: Using non-full backward hooks on a Module that does not take as input a single Tensor or a tuple of Tensors is deprecated and will be removed in future versions. This hook will be missing some of the grad_input. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using non-full backward hooks on a Module that does not take as input a "
/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:939: UserWarning: Using non-full backward h

Finished Epoch 5 || Run Time:    2.1 | Load Time:    3.1 || F1:  45.06 | Prec:  42.79 | Rec:  47.57 || Ex/s: 371.62



tensor(45.0575, device='cuda:0')

#### DIRTY

##### DBLP-ACM

In [81]:
train, validation, test = dm.data.process(
    path='/content/IC/datasesErros/CMedium_30_3_5/Dirty/DBLP-ACM/',
    train='joined_train.csv',
    validation='joined_valid.csv',
    test='joined_test.csv')

In [82]:
model = dm.MatchingModel(attr_summarizer='attention')

In [83]:
model.run_train(
    train,
    validation,
    epochs=15,
    batch_size=32,
    best_save_path='attention_model.pth',
    pos_neg_ratio=3)

* Number of trainable parameters: 4332002
===>  TRAIN Epoch 1


/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:974: UserWarning: Using a non-full backward hook when the forward contains multiple autograd Nodes is deprecated and will be removed in future versions. This hook will be missing some grad_input. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using a non-full backward hook when the forward contains multiple autograd Nodes "
/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:949: UserWarning: Using non-full backward hooks on a Module that does not take as input a single Tensor or a tuple of Tensors is deprecated and will be removed in future versions. This hook will be missing some of the grad_input. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using non-full backward hooks on a Module that does not take as input a "
/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:939: UserWarning: Using non-full backward h

Finished Epoch 1 || Run Time:   16.7 | Load Time:   13.8 || F1:  61.60 | Prec:  55.02 | Rec:  69.97 || Ex/s: 243.25

===>  EVAL Epoch 1


0% [███████████████] 100% | ETA: 00:00:00
Total time elapsed: 00:00:07


Finished Epoch 1 || Run Time:    3.3 | Load Time:    4.4 || F1:  77.07 | Prec:  65.56 | Rec:  93.47 || Ex/s: 324.54

* Best F1: tensor(77.0659, device='cuda:0')
Saving best model...
Done.
---------------------

===>  TRAIN Epoch 2


...

0% [██████████████████████████████] 100% | ETA: 00:00:00
Total time elapsed: 00:00:30


Finished Epoch 15 || Run Time:   16.9 | Load Time:   13.8 || F1:  97.95 | Prec:  97.12 | Rec:  98.80 || Ex/s: 241.69

===>  EVAL Epoch 15


0% [███████████████] 100% | ETA: 00:00:00
Total time elapsed: 00:00:07


Finished Epoch 15 || Run Time:    3.2 | Load Time:    4.3 || F1:  88.77 | Prec:  86.85 | Rec:  90.77 || Ex/s: 326.49

---------------------

Loading best model...
Training done.


tensor(90.5359, device='cuda:0')

In [84]:
# Compute F1 on test set
model.run_eval(test)

===>  EVAL Epoch 8


/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:974: UserWarning: Using a non-full backward hook when the forward contains multiple autograd Nodes is deprecated and will be removed in future versions. This hook will be missing some grad_input. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using a non-full backward hook when the forward contains multiple autograd Nodes "
/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:949: UserWarning: Using non-full backward hooks on a Module that does not take as input a single Tensor or a tuple of Tensors is deprecated and will be removed in future versions. This hook will be missing some of the grad_input. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using non-full backward hooks on a Module that does not take as input a "
/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:939: UserWarning: Using non-full backward h

Finished Epoch 8 || Run Time:    3.3 | Load Time:    4.3 || F1:  90.70 | Prec:  91.32 | Rec:  90.09 || Ex/s: 324.09



tensor(90.7029, device='cuda:0')

##### DBLP-GoogleScholar

In [85]:
train, validation, test = dm.data.process(
    path='/content/IC/datasesErros/CMedium_30_3_5/Dirty/DBLP-GoogleScholar//',
    train='joined_train.csv',
    validation='joined_valid.csv',
    test='joined_test.csv')

In [86]:
model = dm.MatchingModel(attr_summarizer='attention')

In [87]:
model.run_train(
    train,
    validation,
    epochs=15,
    batch_size=32,
    best_save_path='attention_model.pth',
    pos_neg_ratio=3)

* Number of trainable parameters: 4332002
===>  TRAIN Epoch 1


/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:974: UserWarning: Using a non-full backward hook when the forward contains multiple autograd Nodes is deprecated and will be removed in future versions. This hook will be missing some grad_input. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using a non-full backward hook when the forward contains multiple autograd Nodes "
/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:949: UserWarning: Using non-full backward hooks on a Module that does not take as input a single Tensor or a tuple of Tensors is deprecated and will be removed in future versions. This hook will be missing some of the grad_input. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using non-full backward hooks on a Module that does not take as input a "
/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:939: UserWarning: Using non-full backward h

Finished Epoch 1 || Run Time:   37.6 | Load Time:   27.8 || F1:  68.61 | Prec:  60.84 | Rec:  78.64 || Ex/s: 263.24

===>  EVAL Epoch 1


0% [██████████████████████████████] 100% | ETA: 00:00:00
Total time elapsed: 00:00:15


Finished Epoch 1 || Run Time:    7.4 | Load Time:    8.6 || F1:  77.82 | Prec:  69.21 | Rec:  88.88 || Ex/s: 360.53

* Best F1: tensor(77.8232, device='cuda:0')
Saving best model...
Done.
---------------------

===>  TRAIN Epoch 2


...

0% [██████████████████████████████] 100% | ETA: 00:00:00
Total time elapsed: 00:01:05


Finished Epoch 15 || Run Time:   37.6 | Load Time:   28.0 || F1:  98.05 | Prec:  97.27 | Rec:  98.85 || Ex/s: 262.62

===>  EVAL Epoch 15


0% [██████████████████████████████] 100% | ETA: 00:00:00
Total time elapsed: 00:00:16


Finished Epoch 15 || Run Time:    7.4 | Load Time:    8.7 || F1:  83.46 | Prec:  82.29 | Rec:  84.67 || Ex/s: 356.09

---------------------

Loading best model...
Training done.


tensor(84.5471, device='cuda:0')

In [88]:
# Compute F1 on test set
model.run_eval(test)

===>  EVAL Epoch 9


/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:974: UserWarning: Using a non-full backward hook when the forward contains multiple autograd Nodes is deprecated and will be removed in future versions. This hook will be missing some grad_input. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using a non-full backward hook when the forward contains multiple autograd Nodes "
/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:949: UserWarning: Using non-full backward hooks on a Module that does not take as input a single Tensor or a tuple of Tensors is deprecated and will be removed in future versions. This hook will be missing some of the grad_input. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using non-full backward hooks on a Module that does not take as input a "
/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:939: UserWarning: Using non-full backward h

Finished Epoch 9 || Run Time:    7.6 | Load Time:    9.0 || F1:  84.44 | Prec:  80.73 | Rec:  88.50 || Ex/s: 346.03



tensor(84.4405, device='cuda:0')

### HYBRID

#### STRUCTURED

##### Amazon-Google

In [89]:
train, validation, test = dm.data.process(
    path='/content/IC/datasesErros/CMedium_30_3_5/Structured/Amazon-Google/',
    train='joined_train.csv',
    validation='joined_valid.csv',
    test='joined_test.csv')

In [90]:
model = dm.MatchingModel(attr_summarizer='hybrid')

In [91]:
model.run_train(
    train,
    validation,
    epochs=15,
    batch_size=32,
    best_save_path='hybrid_model.pth',
    pos_neg_ratio=3)

/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:949: UserWarning: Using non-full backward hooks on a Module that does not take as input a single Tensor or a tuple of Tensors is deprecated and will be removed in future versions. This hook will be missing some of the grad_input. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using non-full backward hooks on a Module that does not take as input a "
/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:974: UserWarning: Using a non-full backward hook when the forward contains multiple autograd Nodes is deprecated and will be removed in future versions. This hook will be missing some grad_input. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using a non-full backward hook when the forward contains multiple autograd Nodes "


* Number of trainable parameters: 7133105
===>  TRAIN Epoch 1


/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:939: UserWarning: Using non-full backward hooks on a Module that does not return a single Tensor or a tuple of Tensors is deprecated and will be removed in future versions. This hook will be missing some of the grad_output. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using non-full backward hooks on a Module that does not return a "
/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:2742: UserWarning: reduction: 'mean' divides the total loss by both the batch size and the support size.'batchmean' divides only by the batch size, and aligns with the KL div math definition.'mean' will be changed to behave the same as 'batchmean' in the next major release.
  "reduction: 'mean' divides the total loss by both the batch size and the support size."
0% [██████████████████████████████] 100% | ETA: 00:00:00
Total time elapsed: 00:00:22


Finished Epoch 1 || Run Time:   18.3 | Load Time:    4.4 || F1:  23.64 | Prec:  32.42 | Rec:  18.60 || Ex/s: 302.14

===>  EVAL Epoch 1


0% [██████████████] 100% | ETA: 00:00:00
Total time elapsed: 00:00:04


...

0% [██████████████████████████████] 100% | ETA: 00:00:00
Total time elapsed: 00:00:22


Finished Epoch 15 || Run Time:   18.2 | Load Time:    4.4 || F1:  97.40 | Prec:  95.59 | Rec:  99.28 || Ex/s: 303.11

===>  EVAL Epoch 15


0% [██████████████] 100% | ETA: 00:00:00
Total time elapsed: 00:00:04


Finished Epoch 15 || Run Time:    3.5 | Load Time:    1.6 || F1:  40.55 | Prec:  44.00 | Rec:  37.61 || Ex/s: 453.21

---------------------

Loading best model...
Training done.


tensor(44.8696, device='cuda:0')

In [92]:
# Compute F1 on test set
model.run_eval(test)

===>  EVAL Epoch 2


/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:949: UserWarning: Using non-full backward hooks on a Module that does not take as input a single Tensor or a tuple of Tensors is deprecated and will be removed in future versions. This hook will be missing some of the grad_input. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using non-full backward hooks on a Module that does not take as input a "
/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:974: UserWarning: Using a non-full backward hook when the forward contains multiple autograd Nodes is deprecated and will be removed in future versions. This hook will be missing some grad_input. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using a non-full backward hook when the forward contains multiple autograd Nodes "
/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:939: UserWarning: Using non-full backward h

Finished Epoch 2 || Run Time:    3.2 | Load Time:    1.4 || F1:  47.20 | Prec:  39.15 | Rec:  59.40 || Ex/s: 498.29



tensor(47.1986, device='cuda:0')

##### Beer

In [93]:
train, validation, test = dm.data.process(
    path='/content/IC/datasesErros/CMedium_30_3_5/Structured/Beer/',
    train='joined_train.csv',
    validation='joined_valid.csv',
    test='joined_test.csv')

In [94]:
model = dm.MatchingModel(attr_summarizer='hybrid')

In [95]:
model.run_train(
    train,
    validation,
    epochs=15,
    batch_size=32,
    best_save_path='hybrid_model.pth',
    pos_neg_ratio=3)

* Number of trainable parameters: 9210006
===>  TRAIN Epoch 1


/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:949: UserWarning: Using non-full backward hooks on a Module that does not take as input a single Tensor or a tuple of Tensors is deprecated and will be removed in future versions. This hook will be missing some of the grad_input. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using non-full backward hooks on a Module that does not take as input a "
/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:974: UserWarning: Using a non-full backward hook when the forward contains multiple autograd Nodes is deprecated and will be removed in future versions. This hook will be missing some grad_input. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using a non-full backward hook when the forward contains multiple autograd Nodes "
/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:939: UserWarning: Using non-full backward h

Finished Epoch 1 || Run Time:    1.0 | Load Time:    0.3 || F1:  21.67 | Prec:  16.25 | Rec:  32.50 || Ex/s: 202.63

===>  EVAL Epoch 1
Finished Epoch 1 || Run Time:    0.2 | Load Time:    0.1 || F1:  34.57 | Prec:  20.90 | Rec: 100.00 || Ex/s: 342.59

* Best F1: tensor(34.5679, device='cuda:0')
Saving best model...
Done.
---------------------

===>  TRAIN Epoch 2


0% [█] 100% | ETA: 00:00:00
Total time elapsed: 00:00:00


Finished Epoch 2 || Run Time:    1.0 | Load Time:    0.3 || F1:  46.15 | Prec:  41.18 | Rec:  52.50 || Ex/s: 206.02

===>  EVAL Epoch 2
Finished Epoch 2 || Run Time:    0.2 | Load Time:    0.1 || F1:   0.00 | Prec:   0.00 | Rec:   0.00 || Ex/s: 351.03

---------------------

===>  TRAIN Epoch 3


...

0% [█] 100% | ETA: 00:00:00
Total time elapsed: 00:00:00


Finished Epoch 14 || Run Time:    1.0 | Load Time:    0.3 || F1:  97.56 | Prec:  95.24 | Rec: 100.00 || Ex/s: 204.42

===>  EVAL Epoch 14
Finished Epoch 14 || Run Time:    0.2 | Load Time:    0.1 || F1:  62.50 | Prec:  55.56 | Rec:  71.43 || Ex/s: 336.44

---------------------

===>  TRAIN Epoch 15


0% [█] 100% | ETA: 00:00:00
Total time elapsed: 00:00:00


Finished Epoch 15 || Run Time:    1.0 | Load Time:    0.3 || F1:  98.77 | Prec:  97.56 | Rec: 100.00 || Ex/s: 208.74

===>  EVAL Epoch 15
Finished Epoch 15 || Run Time:    0.2 | Load Time:    0.1 || F1:  62.50 | Prec:  55.56 | Rec:  71.43 || Ex/s: 345.94

---------------------

Loading best model...
Training done.


tensor(66.6667, device='cuda:0')

In [96]:
# Compute F1 on test set
model.run_eval(test)

===>  EVAL Epoch 10


/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:949: UserWarning: Using non-full backward hooks on a Module that does not take as input a single Tensor or a tuple of Tensors is deprecated and will be removed in future versions. This hook will be missing some of the grad_input. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using non-full backward hooks on a Module that does not take as input a "
/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:974: UserWarning: Using a non-full backward hook when the forward contains multiple autograd Nodes is deprecated and will be removed in future versions. This hook will be missing some grad_input. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using a non-full backward hook when the forward contains multiple autograd Nodes "
/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:939: UserWarning: Using non-full backward h

Finished Epoch 10 || Run Time:    0.2 | Load Time:    0.1 || F1:  62.86 | Prec:  52.38 | Rec:  78.57 || Ex/s: 320.98



tensor(62.8571, device='cuda:0')

##### DBLP-ACM

In [97]:
train, validation, test = dm.data.process(
    path='/content/IC/datasesErros/CMedium_30_3_5/Structured/DBLP-ACM/',
    train='joined_train.csv',
    validation='joined_valid.csv',
    test='joined_test.csv')

In [98]:
model = dm.MatchingModel(attr_summarizer='hybrid')

In [99]:
model.run_train(
    train,
    validation,
    epochs=15,
    batch_size=32,
    best_save_path='hybrid_model.pth',
    pos_neg_ratio=3)

* Number of trainable parameters: 9210006
===>  TRAIN Epoch 1


/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:949: UserWarning: Using non-full backward hooks on a Module that does not take as input a single Tensor or a tuple of Tensors is deprecated and will be removed in future versions. This hook will be missing some of the grad_input. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using non-full backward hooks on a Module that does not take as input a "
/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:974: UserWarning: Using a non-full backward hook when the forward contains multiple autograd Nodes is deprecated and will be removed in future versions. This hook will be missing some grad_input. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using a non-full backward hook when the forward contains multiple autograd Nodes "
/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:939: UserWarning: Using non-full backward h

Finished Epoch 1 || Run Time:   28.1 | Load Time:   11.0 || F1:  86.37 | Prec:  83.32 | Rec:  89.64 || Ex/s: 189.85

===>  EVAL Epoch 1


0% [███████████████] 100% | ETA: 00:00:00
Total time elapsed: 00:00:08


Finished Epoch 1 || Run Time:    5.0 | Load Time:    3.6 || F1:  97.07 | Prec:  97.07 | Rec:  97.07 || Ex/s: 284.57

* Best F1: tensor(97.0721, device='cuda:0')
Saving best model...
Done.
---------------------

===>  TRAIN Epoch 2


...

0% [██████████████████████████████] 100% | ETA: 00:00:00
Total time elapsed: 00:00:39


Finished Epoch 15 || Run Time:   28.4 | Load Time:   11.0 || F1:  99.74 | Prec:  99.48 | Rec: 100.00 || Ex/s: 188.00

===>  EVAL Epoch 15


0% [███████████████] 100% | ETA: 00:00:00
Total time elapsed: 00:00:08


Finished Epoch 15 || Run Time:    5.0 | Load Time:    3.6 || F1:  98.32 | Prec:  97.77 | Rec:  98.87 || Ex/s: 286.05

---------------------

Loading best model...
Training done.


tensor(98.4305, device='cuda:0')

In [100]:
# Compute F1 on test set
model.run_eval(test)

===>  EVAL Epoch 9


/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:949: UserWarning: Using non-full backward hooks on a Module that does not take as input a single Tensor or a tuple of Tensors is deprecated and will be removed in future versions. This hook will be missing some of the grad_input. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using non-full backward hooks on a Module that does not take as input a "
/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:974: UserWarning: Using a non-full backward hook when the forward contains multiple autograd Nodes is deprecated and will be removed in future versions. This hook will be missing some grad_input. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using a non-full backward hook when the forward contains multiple autograd Nodes "
/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:939: UserWarning: Using non-full backward h

Finished Epoch 9 || Run Time:    5.5 | Load Time:    3.6 || F1:  98.00 | Prec:  96.92 | Rec:  99.10 || Ex/s: 271.91



tensor(97.9955, device='cuda:0')

##### DBLP-GoogleScholar

In [101]:
train, validation, test = dm.data.process(
    path='/content/IC/datasesErros/CMedium_30_3_5/Structured/DBLP-GoogleScholar/',
    train='joined_train.csv',
    validation='joined_valid.csv',
    test='joined_test.csv')

In [102]:
model = dm.MatchingModel(attr_summarizer='hybrid')

In [103]:
model.run_train(
    train,
    validation,
    epochs=15,
    batch_size=32,
    best_save_path='hybrid_model.pth',
    pos_neg_ratio=3)

* Number of trainable parameters: 9210006
===>  TRAIN Epoch 1


/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:949: UserWarning: Using non-full backward hooks on a Module that does not take as input a single Tensor or a tuple of Tensors is deprecated and will be removed in future versions. This hook will be missing some of the grad_input. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using non-full backward hooks on a Module that does not take as input a "
/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:974: UserWarning: Using a non-full backward hook when the forward contains multiple autograd Nodes is deprecated and will be removed in future versions. This hook will be missing some grad_input. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using a non-full backward hook when the forward contains multiple autograd Nodes "
/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:939: UserWarning: Using non-full backward h

Finished Epoch 1 || Run Time:   64.0 | Load Time:   22.6 || F1:  82.16 | Prec:  75.59 | Rec:  89.99 || Ex/s: 198.73

===>  EVAL Epoch 1


0% [██████████████████████████████] 100% | ETA: 00:00:00
Total time elapsed: 00:00:18


Finished Epoch 1 || Run Time:   11.6 | Load Time:    7.3 || F1:  87.51 | Prec:  80.02 | Rec:  96.54 || Ex/s: 303.28

* Best F1: tensor(87.5053, device='cuda:0')
Saving best model...
Done.
---------------------

===>  TRAIN Epoch 2


...

0% [██████████████████████████████] 100% | ETA: 00:00:00
Total time elapsed: 00:01:25


Finished Epoch 15 || Run Time:   63.8 | Load Time:   22.5 || F1:  99.74 | Prec:  99.60 | Rec:  99.88 || Ex/s: 199.60

===>  EVAL Epoch 15


0% [██████████████████████████████] 100% | ETA: 00:00:00
Total time elapsed: 00:00:18


Finished Epoch 15 || Run Time:   11.3 | Load Time:    7.3 || F1:  92.53 | Prec:  92.44 | Rec:  92.62 || Ex/s: 309.80

---------------------

Loading best model...
Training done.


tensor(92.8409, device='cuda:0')

In [104]:
# Compute F1 on test set
model.run_eval(test)

===>  EVAL Epoch 8


/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:949: UserWarning: Using non-full backward hooks on a Module that does not take as input a single Tensor or a tuple of Tensors is deprecated and will be removed in future versions. This hook will be missing some of the grad_input. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using non-full backward hooks on a Module that does not take as input a "
/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:974: UserWarning: Using a non-full backward hook when the forward contains multiple autograd Nodes is deprecated and will be removed in future versions. This hook will be missing some grad_input. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using a non-full backward hook when the forward contains multiple autograd Nodes "
/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:939: UserWarning: Using non-full backward h

Finished Epoch 8 || Run Time:   11.8 | Load Time:    7.5 || F1:  92.56 | Prec:  90.45 | Rec:  94.77 || Ex/s: 297.24



tensor(92.5605, device='cuda:0')

##### Walmart-Amazon

In [105]:
train, validation, test = dm.data.process(
    path='/content/IC/datasesErros/CMedium_30_3_5/Structured/Walmart-Amazon/',
    train='joined_train.csv',
    validation='joined_valid.csv',
    test='joined_test.csv')

In [106]:
model = dm.MatchingModel(attr_summarizer='hybrid')

In [107]:
model.run_train(
    train,
    validation,
    epochs=15,
    batch_size=32,
    best_save_path='hybrid_model.pth',
    pos_neg_ratio=3)

/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:949: UserWarning: Using non-full backward hooks on a Module that does not take as input a single Tensor or a tuple of Tensors is deprecated and will be removed in future versions. This hook will be missing some of the grad_input. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using non-full backward hooks on a Module that does not take as input a "
/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:974: UserWarning: Using a non-full backward hook when the forward contains multiple autograd Nodes is deprecated and will be removed in future versions. This hook will be missing some grad_input. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using a non-full backward hook when the forward contains multiple autograd Nodes "


* Number of trainable parameters: 11286907
===>  TRAIN Epoch 1


/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:939: UserWarning: Using non-full backward hooks on a Module that does not return a single Tensor or a tuple of Tensors is deprecated and will be removed in future versions. This hook will be missing some of the grad_output. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using non-full backward hooks on a Module that does not return a "
/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:2742: UserWarning: reduction: 'mean' divides the total loss by both the batch size and the support size.'batchmean' divides only by the batch size, and aligns with the KL div math definition.'mean' will be changed to behave the same as 'batchmean' in the next major release.
  "reduction: 'mean' divides the total loss by both the batch size and the support size."
0% [██████████████████████████████] 100% | ETA: 00:00:00
Total time elapsed: 00:00:33


Finished Epoch 1 || Run Time:   27.1 | Load Time:    6.4 || F1:  30.20 | Prec:  52.59 | Rec:  21.18 || Ex/s: 183.49

===>  EVAL Epoch 1


0% [█████████████] 100% | ETA: 00:00:00
Total time elapsed: 00:00:06


...

0% [██████████████████████████████] 100% | ETA: 00:00:00
Total time elapsed: 00:00:32


Finished Epoch 15 || Run Time:   26.9 | Load Time:    6.4 || F1:  99.22 | Prec:  99.13 | Rec:  99.31 || Ex/s: 184.57

===>  EVAL Epoch 15


0% [█████████████] 100% | ETA: 00:00:00
Total time elapsed: 00:00:06


Finished Epoch 15 || Run Time:    4.6 | Load Time:    2.0 || F1:  59.73 | Prec:  63.37 | Rec:  56.48 || Ex/s: 310.65

* Best F1: tensor(59.7260, device='cuda:0')
Saving best model...
Done.
---------------------

Loading best model...
Training done.


tensor(59.7260, device='cuda:0')

In [108]:
# Compute F1 on test set
model.run_eval(test)

===>  EVAL Epoch 15


/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:949: UserWarning: Using non-full backward hooks on a Module that does not take as input a single Tensor or a tuple of Tensors is deprecated and will be removed in future versions. This hook will be missing some of the grad_input. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using non-full backward hooks on a Module that does not take as input a "
/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:974: UserWarning: Using a non-full backward hook when the forward contains multiple autograd Nodes is deprecated and will be removed in future versions. This hook will be missing some grad_input. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using a non-full backward hook when the forward contains multiple autograd Nodes "
/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:939: UserWarning: Using non-full backward h

Finished Epoch 15 || Run Time:    4.7 | Load Time:    2.0 || F1:  61.28 | Prec:  66.27 | Rec:  56.99 || Ex/s: 306.47



tensor(61.2813, device='cuda:0')

#### TEXTUAL

##### Abt-Buy

In [109]:
train, validation, test = dm.data.process(
    path='/content/IC/datasesErros/CMedium_30_3_5/Textual/Abt-Buy/',
    train='joined_train.csv',
    validation='joined_valid.csv',
    test='joined_test.csv')

In [110]:
model = dm.MatchingModel(attr_summarizer='hybrid')

In [111]:
model.run_train(
    train,
    validation,
    epochs=15,
    batch_size=32,
    best_save_path='hybrid_model.pth',
    pos_neg_ratio=3)

* Number of trainable parameters: 7133105
===>  TRAIN Epoch 1


/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:949: UserWarning: Using non-full backward hooks on a Module that does not take as input a single Tensor or a tuple of Tensors is deprecated and will be removed in future versions. This hook will be missing some of the grad_input. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using non-full backward hooks on a Module that does not take as input a "
/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:974: UserWarning: Using a non-full backward hook when the forward contains multiple autograd Nodes is deprecated and will be removed in future versions. This hook will be missing some grad_input. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using a non-full backward hook when the forward contains multiple autograd Nodes "
/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:939: UserWarning: Using non-full backward h

Finished Epoch 1 || Run Time:   17.7 | Load Time:    9.5 || F1:  33.18 | Prec:  38.38 | Rec:  29.22 || Ex/s: 211.35

===>  EVAL Epoch 1


0% [████████████] 100% | ETA: 00:00:00
Total time elapsed: 00:00:05


Finished Epoch 1 || Run Time:    3.0 | Load Time:    3.0 || F1:  56.21 | Prec:  54.30 | Rec:  58.25 || Ex/s: 320.31

* Best F1: tensor(56.2061, device='cuda:0')
Saving best model...
Done.
---------------------

===>  TRAIN Epoch 2


...

0% [██████████████████████████████] 100% | ETA: 00:00:00
Total time elapsed: 00:00:27


Finished Epoch 15 || Run Time:   18.1 | Load Time:    9.7 || F1:  97.59 | Prec:  96.51 | Rec:  98.70 || Ex/s: 206.22

===>  EVAL Epoch 15


0% [████████████] 100% | ETA: 00:00:00
Total time elapsed: 00:00:05


Finished Epoch 15 || Run Time:    3.0 | Load Time:    3.0 || F1:  56.63 | Prec:  59.68 | Rec:  53.88 || Ex/s: 320.97

---------------------

Loading best model...
Training done.


tensor(61.9512, device='cuda:0')

In [112]:
# Compute F1 on test set
model.run_eval(test)

===>  EVAL Epoch 3


/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:949: UserWarning: Using non-full backward hooks on a Module that does not take as input a single Tensor or a tuple of Tensors is deprecated and will be removed in future versions. This hook will be missing some of the grad_input. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using non-full backward hooks on a Module that does not take as input a "
/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:974: UserWarning: Using a non-full backward hook when the forward contains multiple autograd Nodes is deprecated and will be removed in future versions. This hook will be missing some grad_input. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using a non-full backward hook when the forward contains multiple autograd Nodes "
/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:939: UserWarning: Using non-full backward h

Finished Epoch 3 || Run Time:    3.4 | Load Time:    3.1 || F1:  64.66 | Prec:  66.84 | Rec:  62.62 || Ex/s: 295.19



tensor(64.6617, device='cuda:0')

#### DIRTY

##### DBLP-ACM

In [113]:
train, validation, test = dm.data.process(
    path='/content/IC/datasesErros/CMedium_30_3_5/Dirty/DBLP-ACM/',
    train='joined_train.csv',
    validation='joined_valid.csv',
    test='joined_test.csv')

In [114]:
model = dm.MatchingModel(attr_summarizer='hybrid')

In [115]:
model.run_train(
    train,
    validation,
    epochs=15,
    batch_size=32,
    best_save_path='hybrid_model.pth',
    pos_neg_ratio=3)

* Number of trainable parameters: 9210006
===>  TRAIN Epoch 1


/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:949: UserWarning: Using non-full backward hooks on a Module that does not take as input a single Tensor or a tuple of Tensors is deprecated and will be removed in future versions. This hook will be missing some of the grad_input. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using non-full backward hooks on a Module that does not take as input a "
/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:974: UserWarning: Using a non-full backward hook when the forward contains multiple autograd Nodes is deprecated and will be removed in future versions. This hook will be missing some grad_input. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using a non-full backward hook when the forward contains multiple autograd Nodes "
/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:939: UserWarning: Using non-full backward h

Finished Epoch 1 || Run Time:   29.3 | Load Time:   13.1 || F1:  63.44 | Prec:  57.08 | Rec:  71.40 || Ex/s: 174.88

===>  EVAL Epoch 1


0% [███████████████] 100% | ETA: 00:00:00
Total time elapsed: 00:00:09


Finished Epoch 1 || Run Time:    5.2 | Load Time:    4.4 || F1:  89.04 | Prec:  92.27 | Rec:  86.04 || Ex/s: 257.27

* Best F1: tensor(89.0443, device='cuda:0')
Saving best model...
Done.
---------------------

===>  TRAIN Epoch 2


...

0% [██████████████████████████████] 100% | ETA: 00:00:00
Total time elapsed: 00:00:42


Finished Epoch 15 || Run Time:   29.5 | Load Time:   13.3 || F1:  99.55 | Prec:  99.25 | Rec:  99.85 || Ex/s: 173.44

===>  EVAL Epoch 15


0% [███████████████] 100% | ETA: 00:00:00
Total time elapsed: 00:00:09


Finished Epoch 15 || Run Time:    5.3 | Load Time:    4.5 || F1:  93.26 | Prec:  93.05 | Rec:  93.47 || Ex/s: 251.25

---------------------

Loading best model...
Training done.


tensor(94.8315, device='cuda:0')

In [116]:
# Compute F1 on test set
model.run_eval(test)

===>  EVAL Epoch 6


/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:949: UserWarning: Using non-full backward hooks on a Module that does not take as input a single Tensor or a tuple of Tensors is deprecated and will be removed in future versions. This hook will be missing some of the grad_input. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using non-full backward hooks on a Module that does not take as input a "
/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:974: UserWarning: Using a non-full backward hook when the forward contains multiple autograd Nodes is deprecated and will be removed in future versions. This hook will be missing some grad_input. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using a non-full backward hook when the forward contains multiple autograd Nodes "
/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:939: UserWarning: Using non-full backward h

Finished Epoch 6 || Run Time:    5.5 | Load Time:    4.5 || F1:  94.53 | Prec:  93.79 | Rec:  95.27 || Ex/s: 245.92



tensor(94.5251, device='cuda:0')

##### DBLP-GoogleScholar

In [117]:
train, validation, test = dm.data.process(
    path='/content/IC/datasesErros/CMedium_30_3_5/Dirty/DBLP-GoogleScholar//',
    train='joined_train.csv',
    validation='joined_valid.csv',
    test='joined_test.csv')

In [118]:
model = dm.MatchingModel(attr_summarizer='hybrid')

In [119]:
model.run_train(
    train,
    validation,
    epochs=15,
    batch_size=32,
    best_save_path='hybrid_model.pth',
    pos_neg_ratio=3)

* Number of trainable parameters: 9210006
===>  TRAIN Epoch 1


/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:949: UserWarning: Using non-full backward hooks on a Module that does not take as input a single Tensor or a tuple of Tensors is deprecated and will be removed in future versions. This hook will be missing some of the grad_input. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using non-full backward hooks on a Module that does not take as input a "
/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:974: UserWarning: Using a non-full backward hook when the forward contains multiple autograd Nodes is deprecated and will be removed in future versions. This hook will be missing some grad_input. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using a non-full backward hook when the forward contains multiple autograd Nodes "
/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:939: UserWarning: Using non-full backward h

Finished Epoch 1 || Run Time:   65.0 | Load Time:   26.6 || F1:  73.33 | Prec:  66.09 | Rec:  82.35 || Ex/s: 187.86

===>  EVAL Epoch 1


0% [██████████████████████████████] 100% | ETA: 00:00:00
Total time elapsed: 00:00:20


Finished Epoch 1 || Run Time:   11.6 | Load Time:    8.6 || F1:  86.05 | Prec:  79.59 | Rec:  93.64 || Ex/s: 283.23

* Best F1: tensor(86.0455, device='cuda:0')
Saving best model...
Done.
---------------------

===>  TRAIN Epoch 2


...

0% [██████████████████████████████] 100% | ETA: 00:00:00
Total time elapsed: 00:01:29


Finished Epoch 15 || Run Time:   63.9 | Load Time:   26.1 || F1:  99.63 | Prec:  99.63 | Rec:  99.63 || Ex/s: 191.34

===>  EVAL Epoch 15


0% [██████████████████████████████] 100% | ETA: 00:00:00
Total time elapsed: 00:00:20


Finished Epoch 15 || Run Time:   11.6 | Load Time:    8.5 || F1:  90.08 | Prec:  88.18 | Rec:  92.06 || Ex/s: 285.42

---------------------

Loading best model...
Training done.


tensor(90.2439, device='cuda:0')

In [120]:
# Compute F1 on test set
model.run_eval(test)

===>  EVAL Epoch 11


/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:949: UserWarning: Using non-full backward hooks on a Module that does not take as input a single Tensor or a tuple of Tensors is deprecated and will be removed in future versions. This hook will be missing some of the grad_input. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using non-full backward hooks on a Module that does not take as input a "
/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:974: UserWarning: Using a non-full backward hook when the forward contains multiple autograd Nodes is deprecated and will be removed in future versions. This hook will be missing some grad_input. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using a non-full backward hook when the forward contains multiple autograd Nodes "
/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:939: UserWarning: Using non-full backward h

Finished Epoch 11 || Run Time:   11.9 | Load Time:    8.7 || F1:  90.39 | Prec:  88.96 | Rec:  91.87 || Ex/s: 279.09



tensor(90.3908, device='cuda:0')

## Step 4. Apply model to new data

### Evaluating on test data
Now that we have a trained model for entity matching, we can now evaluate its accuracy on test data, to estimate the performance of the model on unlabeled data.

In [ ]:
# Compute F1 on test set
'''model.run_eval(test)'''

### Evaluating on unlabeled data

We finally apply the trained model to unlabeled data to get predictions. To do this, we need to first process the unlabeled data.

#### Processing unlabeled data

To process unlabeled data, we use `dm.data.process_unlabeled`, as shown in the code snippet below. The basic parameters for this call are as follows:

* **path (required): ** The full path to the unlabeled data file (not just the directory).
* **trained_model (required): ** The trained model. The model is aware of the configuration of the training data on which it was trained, and so `deepmatcher` reuses the same configuration for the unlabeled data.
* **ignore_columns (optional): ** Any columns in the unlabeled CSV file that you may want to ignore for the purposes of evaluation. If not specified, the columns that were ignored while processing the training set will also be ignored while processing the unlabeled data.

Note that the unlabeled CSV file must have the same schema as the train, validation and test CSVs.

In [ ]:
'''
unlabeled = dm.data.process_unlabeled(
    path='sample_data/itunes-amazon/unlabeled.csv',
    trained_model=model)
'''

#### Obtaining predictions

Next, we call the `run_prediction` method which takes a processed data set object and returns a `pandas` dataframe containing tuple pair IDs (`id` column) and the corresponding match score predictions (`match_score` column). `match_scores` are in [0, 1] and a score above 0.5 indicates a match prediction.

In [ ]:
'''
predictions = model.run_prediction(unlabeled)
predictions.head()
'''

You may optionally set the `output_attributes` parameter to also include all attributes present in the original input table. As mentioned earlier, the processed attribute values will likely look a bit different from the attribute values in the input CSV files due to modifications such as tokenization and lowercasing.

In [ ]:
#predictions = model.run_prediction(unlabeled, output_attributes=True)
#predictions.head()

You can then save these predictions to CSV and use them for downstream tasks.

In [ ]:
#predictions.to_csv('sample_data/itunes-amazon/unlabeled_predictions.csv')

#### Getting predictions on labeled data

You can also get predictions for labeled data such as validation data. To do so, you can simply call the `run_prediction` method passing the validation data as argument.

In [ ]:
#valid_predictions = model.run_prediction(validation, output_attributes=True)
#valid_predictions.head()